In [ ]:
from import_RNA import *

# Gluta

In [ ]:
P22 = sc.read_h5ad('h5ads/sham_S1S2S3B2_analyzed.h5ad')

In [ ]:
P22.X = P22.raw.X

In [ ]:
P22_glut = P22.copy()
for i in ['Astro', 'Endo', 'Lamp5', 'Micro', 'OD', 'OPC', 'Pvalb', 'Sst', 'VLMC', 'Vip']:
    P22_glut = P22_glut[P22_glut.obs.Subclass!=i,:]
pipeline_short(P22_glut, batch_correct=True, batch_ID='Sample',)

In [ ]:
sc.pl.embedding(P22_glut, color=['Subclass', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '15,16', '20,21', '29,30',
                                                                             '30,31'])    

In [ ]:
sc.pl.pca_variance_ratio(P22_glut,)

In [ ]:
sc.pl.umap(shuffle(P22_glut), color=['Sample', 'Subclass'])      

## Res vs F1

In [ ]:
reses = [0.25, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(P22_glut, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(P22_glut, color=['Subclass','leiden_'+str(i)], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
P22_glut.X = P22_glut.raw.X
genes = list(P22_glut[:,P22_glut.var.highly_variable].var_names)
P22_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(P22_glut, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=P22_glut, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(P22_glut, obs))
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    
    subs = []
    for dic in train_dic:
        #print(P22_glut[P22_glut.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(P22_glut[P22_glut.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    #build F1 plot
    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1)) # cluster, F1 score pairs
    f1_ser_subclass = pd.Series(index=subs,data=f1t1) #cluster's-subclass, F1 score pairs
    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    a['Accuracy'] = acc_clusts
    P22_dfs.append(a)
    del a
    #build acc plot
    


In [ ]:
P22_dfs[0]

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(P22_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(P22_dfs), x='Res', y="Accuracy",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

## Map to V1

### P21

In [ ]:
P21 = rem_fem(sc.read_h5ad('P21full_glut_v3.h5ad'))

In [ ]:
sc.pl.umap(P21, color=['Study', 'Subclass', 'Type_nn_dists'], legend_loc='on data')

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P22_glut, adata_t1=P21, test_lab='leiden', train_lab='Type_nn_dists', 
            t0_dict=make_dict(P22_glut, 'leiden'), t1_dict=make_dict(P21, 'Type_nn_dists'),min_cells=200,x_lab='P21 V1',
            y_lab='P22 S1', union_hvgs=False, recomp_HVGs=False)

test_labelsS1vsV1, test_predlabelsS1vsV1, test_prediction_S1vsV1 = test_items
mappingconfmatS1vsV1, mappingxticksS1vsV1, mappingplotS1vsV1 = mapping_items

P22_glut.obs['P21 Mapping Prob'] = np.max(test_prediction_S1vsV1, axis=1)
P22_glut.obs['P21 Mapping Label'] = pd.Categorical(test_predlabelsS1vsV1).rename_categories(list(make_dict(P21, 'Type_nn_dists').keys()))

In [ ]:
sc.pl.umap(shuffle(P22_glut))

In [ ]:
sc.pl.umap(shuffle(P22_glut), color=['Sample', 'Subclass', ])
sc.pl.umap(shuffle(P22_glut), color=['leiden_1', 'leiden_2', 'P21 Mapping Label', 'P21 Mapping Prob'], legend_loc='on data',
          legend_fontsize=10)      

In [ ]:
sc.pl.umap(P22_glut, color=['Cdh13', 'Adamts2', 'Baz1a','Trpc6', 'Agmat','Rrad','Chrm2'], 
           color_map='hot')

In [ ]:
sc.pl.dotplot(P22_glut, var_names=['Cdh13', 'Adamts2', 'Baz1a','Trpc6', 'Agmat','Rrad','Chrm2', 'Mcc'], 
              groupby='P21 Mapping Label')

In [ ]:
for i in reses:
    
    fig, axs = plt.subplots(1,1,figsize=(i*0.8*12, 6))
    hmap_func = make_hmap(P22_glut, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':8},ax=axs,)
    plt.title('ARI = '+str(hmap_func[1])[0:4])
    sc.pl.umap(shuffle(P22_glut), color=['leiden_'+str(i), 'Subclass', 'P21 Mapping Label', 'P21 Mapping Prob'],
               legend_loc='on data',
          legend_fontsize=10)      

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
for i in reses:
    
    hmap_func = make_hmap(P22_glut, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':12},ax=axs[a,b],)
    axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
P22_glut.write_h5ad('h5ads/P22Sham_glut_v1.h5ad')

In [ ]:
P22_glut = sc.read_h5ad('h5ads/P22Sham_glut_v1.h5ad')

## L2/3

In [ ]:
L23 = P22_glut[P22_glut.obs['Subclass']=='L2/3',:].copy()

In [ ]:
sc.pl.violin(L23, 'P21 Mapping Prob','P21 Mapping Label', rotation=90)

In [ ]:
L23.obs['P21 Mapping Label'].value_counts()

In [ ]:
L23.X = L23.raw.X
for i in L23.obs['P21 Mapping Label'].value_counts()[3:].index:
    L23 = L23[L23.obs['P21 Mapping Label']!=i]
pipeline_short(L23, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.embedding(L23, color=['P21 Mapping Label', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '20,21'])    

In [ ]:
sc.pl.pca_variance_ratio(L23,)

del L23.obsp
sc.pp.neighbors(L23, use_rep='X_harmony',n_pcs=20, )
sc.tl.umap(L23)

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(L23, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(L23, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(L23, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
L23.X = L23.raw.X
genes = list(L23[:,L23.var.highly_variable].var_names)
L23_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(L23, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=L23, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(L23, obs))
    subs = []
    for dic in train_dic:
        #print(L23[L23.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(L23[L23.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    a['Accuracy'] = acc_clusts

    L23_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L23_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(L23, color=['leiden_0.3', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Cdh13', 'Adamts2', 'Trpc6', 'Baz1a', 'Agmat', 'Chrm2', 'Ncam2'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=20)      

In [ ]:
sc.pl.embedding(L23, color=['leiden_0.3', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Cdh13', 'Adamts2', 'Trpc6', 'Baz1a', 'Agmat', 'Chrm2'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.settings.set_figure_params()

In [ ]:
sc.pl.dotplot(L23, var_names=['Trpc6', 'Baz1a', 'Npas4', 'Agmat','Chrm2', 'Ncam2', 'Cdh13', 'Adamts2', ],
             groupby='leiden_0.3')

In [ ]:
L23_marks = []
for i in L23.obs['leiden_0.3'].values.categories:
    L23_marks.append(DE(L23,obs_id='leiden_0.3', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))

In [ ]:
'Baz1a' in L23_marks[0].index

In [ ]:
clus_sample_bars(adata=L23, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.3', size=(13,9.5))

In [ ]:
L23.obs['leiden_0.3'].values.categories

In [ ]:
L23.obs['Type'] = L23.obs['leiden_0.3'].cat.rename_categories(new_categories=['L2/3_B', 'L2/3_C', 'L2/3_A'])

In [ ]:
L23.write_h5ad('h5ads/P22Sham_typing/P22Sham_L23_types.h5ad')

In [ ]:
sc.pl.umap(L23, color=['Type', 'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=20)      

In [ ]:
for i in L23_marks:
    print(i.shape)
    sc.pl.dotplot(L23, var_names=i.index[0:20], groupby='Type', dendrogram=False)

In [ ]:
sc.pl.dotplot(L23, var_names=['Fosb', 'Fosl2', 'Nr4a3', 'Egr2',
                             'Nr4a2', 'Nptx2', 'Fos'], 
              groupby='Type', dendrogram=False)

### Condylis et al IEGs in B?

In [ ]:
sc.pl.violin(L23, keys=['Fosb', 'Fosl2', 'Nr4a3', 'Egr2',
                             'Nr4a2', 'Nptx2', 'Fos'], 
              groupby='Type', )

In [ ]:
L23.obs.Type.value_counts()

In [ ]:
for i in L23_marks[2][0:40].index:
    print(i)

## L4

In [ ]:
L4 = P22_glut[P22_glut.obs['Subclass']=='L4',:].copy()
L4.X = L4.raw.X

In [ ]:
L4.obs['P21 Mapping Label'].value_counts()

In [ ]:
sc.pl.violin(L4, 'P21 Mapping Prob','P21 Mapping Label', rotation=90)

In [ ]:
for i in L4.obs['P21 Mapping Label'].value_counts()[3:].index:
    L4 = L4[L4.obs['P21 Mapping Label']!=i]
pipeline_short(L4, batch_correct=True, batch_ID='Sample')

### Res vs F1

In [ ]:
L4 = sc.read_h5ad('h5ads/P22Sham_typing/P22Sham_L4_types.h5ad')

In [ ]:
sc.pl.umap(L4, color=['Bdnf'], legend_loc='on data', 
               legend_fontsize=10)
sc.pl.dotplot(L4, var_names=['Bdnf'], groupby='leiden_0.4')
sc.pl.dotplot(L4, var_names=['Bdnf'], groupby='leiden_0.5')

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(L4, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(L4, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(L4, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
L4.X = L4.raw.X
genes = list(L4[:,L4.var.highly_variable].var_names)
L4_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(L4, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=L4, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(L4, obs))
    subs = []
    for dic in train_dic:
        #print(L4[L4.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(L4[L4.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    L4_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L4_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L23_dfs+L4_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(shuffle(L4), color=['leiden_0.2', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Nell1', 'Etv6', 'Il1rapl2', 
                      'Rspo1', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=20)      

In [ ]:
sc.pl.embedding(L4, color=['leiden_0.2', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Nell1', 'Etv6', 'Il1rapl2', 
                      'Rspo1', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.embedding(L4, color=['leiden_0.2',],
               legend_loc='on data',components=['1,2', '2,3', '3,4', '4,5', '5,6', '6,7'],
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.dotplot(L4, var_names=['Il1rapl2','Nell1', 'Etv6',   'Rspo1', ],
             groupby='leiden_0.3', dendrogram=True)

In [ ]:
L4_marks = []
for i in L4.obs['leiden_0.2'].values.categories:
    L4_marks.append(DE(L4,obs_id='leiden_0.2', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))
for i in L4_marks:
    print(i.shape)
    sc.pl.dotplot(L4, var_names=i.index[0:10], groupby='leiden_0.2', dendrogram=False)

In [ ]:
clus_sample_bars(adata=L4, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.2', size=(13,9.5))

In [ ]:
L4.obs['leiden_0.2'].values.categories

In [ ]:
L4.obs['Type'] = L4.obs['leiden_0.2'].cat.rename_categories(new_categories=['L4_A', 'L4_B', 'L4_C',])

In [ ]:
sc.pl.umap(L4, color=['Type', 'leiden_0.2', 'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=20)      

In [ ]:
L4.write_h5ad('h5ads/P22Sham_typing/P22Sham_L4_types.h5ad')

## Spiny

In [ ]:
L4_V1 = P21[P21.obs.Subclass=='L4',:].copy()
L4_V1 = L4_V1[L4_V1.obs.Study!='2023 Multiome',:].copy()
L4_V1.X = L4_V1.raw.X

In [ ]:
L4_mix = L4.concatenate(L4_V1, batch_categories=['S1', 'V1'], batch_key='Study')

In [ ]:
pipeline_short(L4_mix, batch_correct=False, batch_ID=None)

In [ ]:
sc.pl.umap(L4_mix, color=['Sample', 'Study', 'Type'])

In [ ]:
sc.pl.embedding(L4_mix, color=['Type', 'Study'],
               legend_loc='on data',components=['1,2', '2,3', '4,5', '5,6'],
          legend_fontsize=10, color_map='hot', basis='X_pca')    

In [ ]:
L4_mix.X = L4_mix.raw.X
pipeline_short(L4_mix, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.umap(shuffle(L4_mix), color=['Sample', 'Study', 'Type', 'leiden'])

In [ ]:
sc.pl.embedding(L4_mix, color=['Type', 'Study'],
               legend_loc='on data',components=['1,2', '2,3', '4,5', '5,6', '6,7', '7,8', '8,9', '9,10'],
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
clus_sample_bars(adata=L4_mix, a_=4, b_=3, samp_id='Study', clus_id='leiden', size=(13,15))

## L6CT

In [ ]:
L6CT = P22_glut[P22_glut.obs['Subclass']=='L6CT',:].copy()
L6CT.X = L6CT.raw.X
L6CT.obs['P21 Mapping Label'].value_counts()

In [ ]:
for i in L6CT.obs['P21 Mapping Label'].value_counts()[3:].index:
    L6CT = L6CT[L6CT.obs['P21 Mapping Label']!=i]
pipeline_short(L6CT, batch_correct=True, batch_ID='Sample')

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(L6CT, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(L6CT, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(L6CT, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
L6CT.X = L6CT.raw.X
genes = list(L6CT[:,L6CT.var.highly_variable].var_names)
L6CT_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(L6CT, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=L6CT, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(L6CT, obs))
    subs = []
    for dic in train_dic:
        #print(L6CT[L6CT.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(L6CT[L6CT.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    L6CT_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L23_dfs+L4_dfs+L6CT_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(L6CT, color=['leiden_0.3', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Galntl6', 'Brinp3', 'Htr4'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=20)      

In [ ]:
sc.pl.embedding(L6CT, color=['leiden_0.3', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob','Galntl6', 'Brinp3', 'Htr4'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.embedding(L6CT, color=['leiden_0.3',],
               legend_loc='on data',components=['1,2', '2,3', '3,4', '4,5'],
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

L6CT = L6CT[L6CT.obs['leiden_0.3']!='3',:]
L6CT = L6CT[L6CT.obs['leiden_0.3']!='4',:]

In [ ]:
sc.pl.dotplot(L6CT, var_names=[ 'Brinp3', 'Htr4','Galntl6',],
             groupby='leiden_0.3')

In [ ]:
L6CT_marks = []
for i in L6CT.obs['leiden_0.3'].values.categories:
    L6CT_marks.append(DE(L6CT,obs_id='leiden_0.3', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))
for i in L6CT_marks:
    print(i.shape)
    sc.pl.dotplot(L6CT, var_names=i.index[0:10], groupby='leiden_0.3', dendrogram=False)

In [ ]:
clust_obs_plot(L6CT, 'P21 Mapping Prob', obs_id_split='leiden_0.3')
clust_obs_plot(L6CT, 'pct_counts_mt', obs_id_split='leiden_0.3')
clust_obs_plot(L6CT, 'Doublet Score', obs_id_split='leiden_0.3')
clust_obs_plot(L6CT, 'total_counts', obs_id_split='leiden_0.3')

In [ ]:
clus_sample_bars(adata=L6CT, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.3', size=(13,9.5))

In [ ]:
L6CT.obs['leiden_0.3'].values.categories

In [ ]:
L6CT.obs['Type'] = L6CT.obs['leiden_0.3'].cat.rename_categories(new_categories=['L6CT_B', 'L6CT_C', 'L6CT_A', 
                                                                                'L6CT_D', 'L6CT_E'])

In [ ]:
sc.pl.umap(L6CT, color=['Type', 'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=20)      

In [ ]:
L6CT.write_h5ad('h5ads/P22Sham_typing/P22Sham_L6CT_types.h5ad')

In [ ]:
L6CT = sc.read_h5ad('h5ads/P22Sham_typing/P22Sham_L6CT_types.h5ad')

In [ ]:
sc.pl.dotplot(L6CT, var_names=['Arhgap31', 'Pdlim1', 'Fam241a', 'Brinp3', 'Ust', 'Nell1',], groupby='Type')

## L56IT

In [ ]:
L56IT = P22_glut[P22_glut.obs['Subclass']=='L5IT',:].concatenate(P22_glut[P22_glut.obs['Subclass']=='L6IT',:]).copy()
L56IT.X = L56IT.raw.X
L56IT.obs['P21 Mapping Label'].value_counts()

In [ ]:
for i in L56IT.obs['P21 Mapping Label'].value_counts()[3:].index:
    L56IT = L56IT[L56IT.obs['P21 Mapping Label']!=i]
pipeline_short(L56IT, batch_correct=True, batch_ID='Sample')

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(L56IT, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(L56IT, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
L56IT = L56IT[L56IT.obs['leiden_0.4']!='5',:]

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(L56IT, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(L56IT, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(L56IT, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
L56IT.X = L56IT.raw.X
genes = list(L56IT[:,L56IT.var.highly_variable].var_names)
L56IT_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(L56IT, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=L56IT, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(L56IT, obs))
    subs = []
    for dic in train_dic:
        #print(L56IT[L56IT.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(L56IT[L56IT.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    L56IT_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L56IT_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L23_dfs+L4_dfs+L56IT_dfs+L6CT_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(L56IT, color=['leiden_0.4', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Deptor', 'Etv1', 'Pparg', 'Pld5'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

In [ ]:
sc.pl.embedding(L56IT, color=['leiden_0.4', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob','Deptor', 'Etv1', 'Pparg', 'Pld5'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.embedding(L56IT, color=['leiden_0.4',],
               legend_loc='on data',components=['1,2', '2,3', '3,4', '4,5', '5,6'],
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.dotplot(L56IT, var_names=['Deptor', 'Etv1', 'Pparg', 'Pld5'],
             groupby='leiden_0.4')

In [ ]:
L56IT_marks = []
for i in L56IT.obs['leiden_0.4'].values.categories:
    L56IT_marks.append(DE(L56IT,obs_id='leiden_0.4', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))

In [ ]:
for i in L56IT_marks:
    print(i.shape)
    sc.pl.dotplot(L56IT, var_names=i.index[0:10], groupby='leiden_0.4', dendrogram=False)

In [ ]:
clus_sample_bars(adata=L56IT, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.4', size=(13,9.5))

In [ ]:
L56IT.obs['leiden_0.4'].cat.categories

In [ ]:
L56IT.obs['Type'] = L56IT.obs['leiden_0.4'].cat.rename_categories(new_categories=['L5IT', 'L6IT_A', 
                                                                                  'L6IT_B', 'L6IT_Car3',
                                                                                  'L6IT_C'])

In [ ]:
sc.pl.umap(L56IT, color=['Type', 'leiden_0.4', 'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

In [ ]:
L56IT.write_h5ad('h5ads/P22Sham_typing/P22Sham_L56IT_types.h5ad')

## L5NP/PT

In [ ]:
L5NPT = P22_glut[P22_glut.obs['Subclass']=='L5NP',:].concatenate(P22_glut[P22_glut.obs['Subclass']=='L5PT',:]).copy()
L5NPT.X = L5NPT.raw.X
L5NPT.obs['P21 Mapping Label'].value_counts()

In [ ]:
for i in L5NPT.obs['P21 Mapping Label'].value_counts()[3:].index:
    L5NPT = L5NPT[L5NPT.obs['P21 Mapping Label']!=i]
pipeline_short(L5NPT, batch_correct=True, batch_ID='Sample')

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(L5NPT, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(L5NPT, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(L5NPT, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
L5NPT.X = L5NPT.raw.X
genes = list(L5NPT[:,L5NPT.var.highly_variable].var_names)
L5NPT_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(L5NPT, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=L5NPT, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(L5NPT, obs))
    subs = []
    for dic in train_dic:
        #print(L5NPT[L5NPT.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(L5NPT[L5NPT.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    L5NPT_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L5NPT_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(L23_dfs+L4_dfs+L5NPT_dfs+L56IT_dfs+L6CT_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(L5NPT, color=['leiden_0.7', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', 'Tshz2', 'Qrfpr', 'Fn1'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=40)      

In [ ]:
sc.pl.embedding(L5NPT, color=['leiden_0.7', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',  'Tshz2', 'Qrfpr', 'Fn1'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.embedding(L56IT, color=['leiden_0.7',],
               legend_loc='on data',components=['1,2', '2,3', '3,4', '4,5', '5,6'],
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
sc.pl.dotplot(L5NPT, var_names=[ 'Tshz2', 'Qrfpr', 'Fn1', ],
             groupby='leiden_0.7')

In [ ]:
sc.tl.dendrogram(L5NPT, groupby='leiden_0.7')

In [ ]:
L5NPT_marks = []
for i in L5NPT.uns["dendrogram_['leiden_0.7']"]['categories_ordered']:
    L5NPT_marks.append(DE(L5NPT,obs_id='leiden_0.7', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))

They just have to be DE within each dendrogram clade

In [ ]:
for i in L5NPT_marks:
    print(i.shape)
    sc.pl.dotplot(L5NPT, var_names=i.index[0:10], groupby='leiden_0.7', dendrogram=True)

In [ ]:
clus_sample_bars(adata=L5NPT, a_=4, b_=3, samp_id='Sample', clus_id='leiden_0.7', size=(13,11))

In [ ]:
L5NPT.obs['leiden_0.7'].values.categories

In [ ]:
sc.pl.umap(L5NPT, color=['leiden_0.7', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', size=40)      

In [ ]:
news = ['L5NP_A', 'L5PT_A', 'L5PT_B', 'L5PT_C', 'L5NP_B', 'L5PT_D', 'L5PT_E', 'L5PT_F', 'L5PT_G', 'L5NP_C']
L5NPT.obs['Type'] = L5NPT.obs['leiden_0.7'].cat.rename_categories(new_categories=news)

In [ ]:
sc.pl.umap(L5NPT, color=['Type', 'P21 Mapping Label', 'leiden_0.7'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

In [ ]:
L5NPT.write_h5ad('h5ads/P22Sham_typing/P22Sham_L5NPT_types.h5ad')

## Finalize Type assignments

Removed during subclass-specific cell typing:

**A few hundred cells in every section that were mapping to something else when mapped to P21**

In [ ]:
L6b = P22_glut[P22_glut.obs.Subclass=='L6b',:]
L6b.obs['Type'] = pd.Categorical(L6b.shape[0]*['L6b'])

In [ ]:
P22_glut_types_list = []
for i in ['P22Sham_L23_types', 'P22Sham_L4_types', 'P22Sham_L56IT_types', 'P22Sham_L5NPT_types',
         'P22Sham_L6CT_types']:
    P22_glut_types_list.append(sc.read_h5ad('h5ads/P22Sham_typing/'+i+'.h5ad'))

a = P22_glut_types_list+[L6b]

P22_glut_types = a[0].concatenate(a[1:],
                                  batch_categories=None)

In [ ]:
P22_glut_types.shape, P22_glut.shape

In [ ]:
pipeline_short(P22_glut_types, True, 'Sample')

In [ ]:
P22_glut_types.uns['Type_colors'] = ['#a1c9f4', '#ffb482', '#8de5a1', '#ff9f9b', 
                                     '#d0bbff', '#debb9b', '#fab0e4', '#cfcfcf',
                                     '#fffea3', '#b9f2f0', 
                                    '#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', 
                                     '#a6d854', '#ffd92f', '#e5c494', '#b3b3b3',
                                    '#f77189', '#bb9832', '#50b131', '#36ada4', '#3ba3ec', '#e866f4',
                                    
                                    '#f77189', '#bb9832', '#50b131', '#36ada4', '#3ba3ec', '#e866f4']

In [ ]:
sc.pl.umap(shuffle(P22_glut_types), color=['Sample'],  )
sc.pl.umap(P22_glut_types, color=['Type', 'Subclass', 'leiden'], legend_loc='on data', legend_fontsize=10)

In [ ]:
sc.pl.correlation_matrix(L5NPT, groupby='Type',show_correlation_numbers=True)

In [ ]:
plt.rcParams['font.size'] = 11
sc.pl.correlation_matrix(P22_glut_types, groupby='Type',show_correlation_numbers=True)

In [ ]:
P22_glut_types.write_h5ad('h5ads/P22Sham_glut_v2.h5ad')

In [ ]:
P22_glut_types = sc.read_h5ad('h5ads/P22Sham_glut_v2.h5ad')

In [ ]:
FXR1P

In [ ]:
FXR1P, and FXR2P

# GABA

In [ ]:
P22.obs.Subclass.values.categories

In [ ]:
P22_gaba = P22.copy()
for i in ['Astro', 'Endo', 'L2/3', 'L4', 'L5IT', 'L5NP', 'L5PT', 'L6CT', 'L6IT',
       'L6b', 'Micro', 'OD', 'OPC', 'VLMC']:
    P22_gaba = P22_gaba[P22_gaba.obs.Subclass!=i,:]
pipeline_short(P22_gaba, batch_correct=True, batch_ID='Sample',)

In [ ]:
sc.pl.embedding(P22_gaba, color=['Subclass', 'Fos', 'Egr1'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '15,16', '20,21', '29,30', '30,31'])
   

In [ ]:
sc.pl.violin(P22, keys=['Fos', 'Egr1', 'Pvalb', 'Cux2', 'Ccbe1'], groupby='Subclass', rotation=90)

In [ ]:
sc.pl.embedding(P22_gaba, color=['Subclass', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '15,16', '20,21', '29,30', '30,31'])
   

In [ ]:
sc.pl.umap(P22_gaba, color=['Sample', 'Subclass', 'Meis2', 'Pax6', 'Ccnd1',
                           'Doublet Score', 'pct_counts_mt'])      

## Res vs F1

In [ ]:
reses = [0.25, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(P22_gaba, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(P22_gaba, color=['Subclass','leiden_'+str(i)], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
P22_gaba.X = P22_gaba.raw.X
genes = list(P22_gaba[:,P22_gaba.var.highly_variable].var_names)
P22_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(P22_gaba, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=P22_gaba, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(P22_gaba, obs))
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    
    subs = []
    for dic in train_dic:
        #print(P22_gaba[P22_gaba.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(P22_gaba[P22_gaba.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    #build F1 plot
    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1)) # cluster, F1 score pairs
    f1_ser_subclass = pd.Series(index=subs,data=f1t1) #cluster's-subclass, F1 score pairs
    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    a['Accuracy'] = acc_clusts
    P22_dfs.append(a)
    del a
    #build acc plot
    


In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(P22_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(P22_dfs), x='Res', y="Accuracy",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

## Map to V1

### P21

In [ ]:
P21 = rem_fem(sc.read_h5ad('P21full_gaba_v3.h5ad'))

In [ ]:
sc.pl.umap(P21, color=['Study', 'Subclass', 'Type_nn_dists'], legend_loc='on data')

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P22_gaba, adata_t1=P21, test_lab='leiden', train_lab='Type_nn_dists', 
            t0_dict=make_dict(P22_gaba, 'leiden'), t1_dict=make_dict(P21, 'Type_nn_dists'),min_cells=200,x_lab='P21 V1',
            y_lab='P22 S1', union_hvgs=False, recomp_HVGs=False)

test_labelsS1vsV1, test_predlabelsS1vsV1, test_prediction_S1vsV1 = test_items
mappingconfmatS1vsV1, mappingxticksS1vsV1, mappingplotS1vsV1 = mapping_items

P22_gaba.obs['P21 Mapping Prob'] = np.max(test_prediction_S1vsV1, axis=1)
P22_gaba.obs['P21 Mapping Label'] = pd.Categorical(test_predlabelsS1vsV1).rename_categories(list(make_dict(P21, 'Type_nn_dists').keys()))

In [ ]:
sc.pl.umap(shuffle(P22_gaba), color=['Sample', 'Subclass', ])
sc.pl.umap(shuffle(P22_gaba), color=['leiden_1', 'leiden_2', 'P21 Mapping Label', 'P21 Mapping Prob'], legend_loc='on data',
          legend_fontsize=10)      

In [ ]:
for i in reses:
    
    fig, axs = plt.subplots(1,1,figsize=(i*0.8*12, 6))
    hmap_func = make_hmap(P22_gaba, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':8},ax=axs,)
    plt.title('ARI = '+str(hmap_func[1])[0:4])
    sc.pl.umap(shuffle(P22_gaba), color=['leiden_'+str(i), 'Subclass', 'P21 Mapping Label', 'P21 Mapping Prob'],
               legend_loc='on data',
          legend_fontsize=10)      

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
for i in reses:
    
    hmap_func = make_hmap(P22_gaba, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':12},ax=axs[a,b],)
    axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
P22_gaba.write_h5ad('h5ads/P22Sham_gaba_v1.h5ad')

## Pvalb

In [ ]:
Pvalb = P22_gaba[P22_gaba.obs['Subclass']=='Pvalb',:].copy()

In [ ]:
sc.pl.violin(Pvalb, 'P21 Mapping Prob','P21 Mapping Label', rotation=90)

In [ ]:
Pvalb.obs['P21 Mapping Label'].value_counts()

In [ ]:
Pvalb.obs['P21 Mapping Label'].value_counts()[4:].index

In [ ]:
Pvalb.X = Pvalb.raw.X
for i in Pvalb.obs['P21 Mapping Label'].value_counts()[4:].index:
    Pvalb = Pvalb[Pvalb.obs['P21 Mapping Label']!=i]
pipeline_short(Pvalb, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.embedding(Pvalb, color=['P21 Mapping Label', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '20,21'])    

In [ ]:
sc.pl.pca_variance_ratio(Pvalb,)

del Pvalb.obsp
sc.pp.neighbors(Pvalb, use_rep='X_harmony',n_pcs=20, )
sc.tl.umap(Pvalb)

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(Pvalb, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(Pvalb, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(Pvalb, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
Pvalb = sc.read_h5ad('h5ads/P22Sham_typing/P22Sham_Pvalb_types.h5ad')

In [ ]:
Pvalb.X = Pvalb.raw.X
genes = list(Pvalb[:,Pvalb.var.highly_variable].var_names)
Pvalb_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(Pvalb, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=Pvalb, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(Pvalb, obs))
    subs = []
    for dic in train_dic:
        #print(Pvalb[Pvalb.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(Pvalb[Pvalb.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    a['Accuracy'] = acc_clusts

    Pvalb_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Pvalb_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(Pvalb, color=['leiden_0.4', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

In [ ]:
sc.pl.embedding(Pvalb, color=['leiden_0.4', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
Pvalb_marks = []
for i in Pvalb.obs['leiden_0.4'].values.categories:
    Pvalb_marks.append(DE(Pvalb,obs_id='leiden_0.4', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))

In [ ]:
for i in Pvalb_marks:
    print(i.shape)
    sc.pl.dotplot(Pvalb, var_names=i.index[0:10], groupby='leiden_0.4', dendrogram=True)

In [ ]:
sc.pl.dotplot(Pvalb, var_names=['Pvalb', 'Vipr2', 'Tpbg', 'Reln', 'Tac1'], groupby='leiden_0.4', dendrogram=True)

In [ ]:
clus_sample_bars(adata=Pvalb, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.4', size=(13,9.5))

In [ ]:
Pvalb.obs['leiden_0.4'].values.categories

In [ ]:
Pvalb.obs['Type'] = Pvalb.obs['leiden_0.4'].cat.rename_categories(new_categories=['Pvalb_A', 
                                                                                  'Pvalb_B',
                                                                                  'Pvalb_D',
                                                                                 'Pvalb_C', 
                                                                                 'Pvalb_E',
                                                                                 'Pvalb_F'])

In [ ]:
Pvalb.write_h5ad('h5ads/P22Sham_typing/P22Sham_Pvalb_types.h5ad')

In [ ]:
sc.pl.umap(Pvalb, color=['Type','P21 Mapping Label', 'leiden_0.4'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot')      

In [ ]:
for i in Pvalb_marks:
    print(i.shape)
    sc.pl.dotplot(Pvalb, var_names=i.index[0:20], groupby='Type', dendrogram=False)

In [ ]:
sc.pl.dotplot(Pvalb, var_names=['Pvalb', 'Vipr2', 'Tpbg', 'Reln', 'Tac1'], groupby='Type', dendrogram=True)

In [ ]:
Pvalb.obs.Type.value_counts()

In [ ]:
for i in Pvalb_marks[-2][0:40].index:
    print(i)

## Sst

In [ ]:
P22_gaba = sc.read_h5ad('h5ads/P22Sham_gaba_v1.h5ad')

In [ ]:
Sst = P22_gaba[P22_gaba.obs['Subclass']=='Sst',:].copy()

In [ ]:
sc.pl.violin(Sst, 'P21 Mapping Prob','P21 Mapping Label', rotation=90)

In [ ]:
Sst.obs['P21 Mapping Label'].value_counts()

In [ ]:
Sst.X = Sst.raw.X
for i in Sst.obs['P21 Mapping Label'].value_counts()[5:].index:
    Sst = Sst[Sst.obs['P21 Mapping Label']!=i]
pipeline_short(Sst, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.embedding(Sst, color=['P21 Mapping Label', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '20,21'])    

In [ ]:
sc.pl.pca_variance_ratio(Sst,)

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(Sst, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(Sst, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(Sst, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
Sst.X = Sst.raw.X
genes = list(Sst[:,Sst.var.highly_variable].var_names)
Sst_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(Sst, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=Sst, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(Sst, obs))
    subs = []
    for dic in train_dic:
        #print(Sst[Sst.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(Sst[Sst.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    a['Accuracy'] = acc_clusts

    Sst_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Sst_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Pvalb_dfs+Sst_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

0.7 to 1.0 all give the same clustering, so choose 0.9 b/c it has the highest accuracy

In [ ]:
sc.pl.umap(Sst, color=['leiden_0.9', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot')      

In [ ]:
sc.pl.embedding(Sst, color=['leiden_0.9', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
Sst_marks = []
for i in Sst.obs['leiden_0.9'].values.categories:
    Sst_marks.append(DE(Sst,obs_id='leiden_0.9', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))

In [ ]:
for i in Sst_marks:
    sc.pl.dotplot(Sst, var_names=i.index[0:30], groupby='leiden_0.9', )

In [ ]:
sc.pl.umap(Sst, color=['leiden_0.9', 'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

In [ ]:
clus_sample_bars(adata=Sst, a_=4, b_=3, samp_id='Sample', clus_id='leiden_0.9', size=(13,13))

In [ ]:
Sst.obs['leiden_0.9'].values.categories

In [ ]:
Sst.obs['Type'] = Sst.obs['leiden_0.9'].cat.rename_categories(new_categories=['Sst_A', 'Sst_B', 'Sst_C', 'Sst_D', 
                                                                              'Sst_E', 'Sst_F', 'Sst_G', 'Sst_H', 
                                                                              'Sst_I', 'Sst_J'])

In [ ]:
Sst.write_h5ad('h5ads/P22Sham_typing/P22Sham_Sst_types.h5ad')

In [ ]:
sc.pl.umap(Sst, color=['Type','P21 Mapping Label', 'leiden_0.9'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

## Vip

In [ ]:
Vip = P22_gaba[P22_gaba.obs['Subclass']=='Vip',:].copy()

In [ ]:
sc.pl.violin(Vip, 'P21 Mapping Prob','P21 Mapping Label', rotation=90)

In [ ]:
Vip.obs['P21 Mapping Label'].value_counts()

In [ ]:
Vip.X = Vip.raw.X
for i in Vip.obs['P21 Mapping Label'].value_counts()[5:].index:
    Vip = Vip[Vip.obs['P21 Mapping Label']!=i]
pipeline_short(Vip, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.embedding(Vip, color=['P21 Mapping Label', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '20,21'])    

In [ ]:
sc.pl.pca_variance_ratio(Vip,)

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(Vip, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(Vip, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
sc.pl.umap(Vip, color=['Subclass','leiden_0.2', 'P21 Mapping Label', 'P21 Mapping Prob'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(Vip, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
Vip.X = Vip.raw.X
genes = list(Vip[:,Vip.var.highly_variable].var_names)
Vip_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(Vip, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=Vip, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(Vip, obs))
    subs = []
    for dic in train_dic:
        #print(Vip[Vip.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(Vip[Vip.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    a['Accuracy'] = acc_clusts

    Vip_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Vip_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Pvalb_dfs+Sst_dfs+Vip_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(Vip, color=['leiden_0.2', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot')      

In [ ]:
sc.pl.embedding(Vip, color=['leiden_0.2', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
Vip_marks = []
for i in Vip.obs['leiden_0.2'].values.categories:
    Vip_marks.append(DE(Vip,obs_id='leiden_0.2', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))
    
for i in Vip_marks:
    sc.pl.dotplot(Vip, var_names=i.index[0:30], groupby='leiden_0.2', )

In [ ]:
clus_sample_bars(adata=Vip, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.2', size=(13,9.5))

In [ ]:
Vip.obs['leiden_0.2'].values.categories

In [ ]:
Vip.obs['Type'] = Vip.obs['leiden_0.2'].cat.rename_categories(new_categories=['Vip_A', 'Vip_B', 'Vip_C', 
                                                                              'Vip_D', 'Vip_E', 'Vip_F'])

In [ ]:
Vip.write_h5ad('h5ads/P22Sham_typing/P22Sham_Vip_types.h5ad')

In [ ]:
sc.pl.umap(Vip, color=['Type','leiden_0.2', 'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot')      

In [ ]:
Vip = sc.read_h5ad('h5ads/P22Sham_typing/P22Sham_Vip_types.h5ad')

In [ ]:
sc.pl.dotplot(Vip, var_names=['Chat'], groupby='leiden_0.2', )

## Lamp5

In [ ]:
Lamp5 = P22_gaba[P22_gaba.obs['Subclass']=='Lamp5',:].copy()

In [ ]:
sc.pl.violin(Lamp5, 'P21 Mapping Prob','P21 Mapping Label', rotation=90)

In [ ]:
Lamp5.obs['P21 Mapping Label'].value_counts()

In [ ]:
Lamp5.X = Lamp5.raw.X
for i in Lamp5.obs['P21 Mapping Label'].value_counts()[2:].index:
    Lamp5 = Lamp5[Lamp5.obs['P21 Mapping Label']!=i]
pipeline_short(Lamp5, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.embedding(Lamp5, color=['P21 Mapping Label', ],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony', components=['1,2', '2,3', '3,4',
                                                                             '4,5','5,6','6,7', '7,8', '8,9',
                                                                             '20,21'])    

In [ ]:
sc.pl.pca_variance_ratio(Lamp5,)

### Res vs F1

In [ ]:
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    sc.tl.leiden(Lamp5, resolution=i, key_added='leiden_'+str(i))
    sc.pl.umap(Lamp5, color=['Subclass','leiden_'+str(i), 'P21 Mapping Label'], legend_loc='on data', 
               legend_fontsize=10)

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(30, 20))
a,b=0,0
reses = [0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2]
for i in reses:
    
    hmap_func = make_hmap(Lamp5, thres=0.01, y_obs='P21 Mapping Label', x_obs='leiden_'+str(i))

    sn.heatmap(hmap_func[0], 
                   xticklabels=True, yticklabels=True, cmap='Reds',annot=True, 
                   annot_kws={'size':10},ax=axs[a,b],)
    #axs[a,b].set_title('ARI = '+str(hmap_func[1])[0:4])
    b = b +1
    
    if (b>2):
        b = 0
        a = a + 1
plt.tight_layout()

In [ ]:
Lamp5.X = Lamp5.raw.X
genes = list(Lamp5[:,Lamp5.var.highly_variable].var_names)
Lamp5_dfs = []
for i in reses:
    obs = 'leiden_'+str(i)
    train_dic = make_dict(Lamp5, obs)
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1, f1t1 = trainclassifier(train_anndata=Lamp5, 
                                                                                                     common_top_genes=genes, 
                                                                                                     obs_id=obs, 
                                                                                                     train_dict=train_dic, 
                                                                                                     eta=0.2,
                                                                                                     max_cells_per_ident=1000, 
                                                                                              train_frac=0.7, 
                                                                                                     min_cells_per_ident=100)
    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=make_dict(Lamp5, obs))
    subs = []
    for dic in train_dic:
        #print(Lamp5[Lamp5.obs[obs]==i,:].obs['Subclass'].value_counts().index[0])
        subs.append(Lamp5[Lamp5.obs[obs]==dic,:].obs['Subclass'].value_counts().index[0])

    f1_dict_leiden = dict(zip(train_dic.keys(), f1t1))
    f1_ser_subclass = pd.Series(index=subs,data=f1t1)

    a = pd.DataFrame(f1_ser_subclass)
    a['F1'] = a.values
    a['Subclass'] = a.index
    a['Res'] = a.shape[0]*[i]
    
    true = pd.Categorical(validation_label_train_70t0vst1)
    pred = pd.Categorical(valid_predlabels_train_70t0vst1)

    matrix = confusion_matrix(true, pred)
    acc_clusts = matrix.diagonal()/matrix.sum(axis=1)
    
    a['Accuracy'] = acc_clusts

    Lamp5_dfs.append(a)
    del a

In [ ]:
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Lamp5_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
#plt.figure(figsize=(7,5))
#plt.figure(figsize=(7,5))
sn.pointplot(data=pd.concat(Pvalb_dfs+Sst_dfs+Vip_dfs+Lamp5_dfs), x='Res', y="F1",hue='Subclass',capsize=.1, errwidth=1,)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1,1))

### Typing

In [ ]:
sc.pl.umap(Lamp5, color=['leiden_0.2', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot',)      

In [ ]:
sc.pl.embedding(Lamp5, color=['leiden_0.2', 'Sample','Subclass', 'P21 Mapping Label', 
                        'P21 Mapping Prob',],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot', basis='X_harmony')    

In [ ]:
Lamp5_marks = []
for i in Lamp5.obs['leiden_0.2'].values.categories:
    Lamp5_marks.append(DE(Lamp5,obs_id='leiden_0.2', obs_id_test=i, ref='rest', pts_thresh=0.2, lf_thresh=0.6))
for i in Lamp5_marks:
    sc.pl.dotplot(Lamp5, var_names=i.index[0:30], groupby='leiden_0.2')

In [ ]:
clus_sample_bars(adata=Lamp5, a_=3, b_=3, samp_id='Sample', clus_id='leiden_0.2', size=(13,9.5))

In [ ]:
Lamp5.obs['leiden_0.2'].values.categories

In [ ]:
Lamp5.obs['Type'] = Lamp5.obs['leiden_0.2'].cat.rename_categories(new_categories=['Lamp5_A', 'Lamp5_B', 'Lamp5_C',
                                                                                 'Lamp5_D'])

In [ ]:
Lamp5.write_h5ad('h5ads/P22Sham_typing/P22Sham_Lamp5_types.h5ad')

In [ ]:
sc.pl.umap(Lamp5, color=['Type','leiden_0.2',  'P21 Mapping Label'],
               legend_loc='on data',
          legend_fontsize=10, color_map='hot')      

## Finalize Type assignments

Removed during subclass-specific cell typing:

**A few hundred cells in every section that were mapping to something else when mapped to P21**

In [ ]:
P22_gaba_types_list = []
for i in ['P22Sham_Pvalb_types', 'P22Sham_Sst_types', 'P22Sham_Vip_types', 'P22Sham_Lamp5_types',]:
    P22_gaba_types_list.append(sc.read_h5ad('h5ads/P22Sham_typing/'+i+'.h5ad'))


P22_gaba_types = P22_gaba_types_list[0].concatenate(P22_gaba_types_list[1:],
                                  batch_categories=None)

In [ ]:
P22_gaba_types.shape, P22_gaba.shape

In [ ]:
print('hi')

In [ ]:
pipeline_short(P22_gaba_types, True, 'Sample')

In [ ]:
P22_gaba_types.uns['Type_colors'] = ['#a1c9f4', '#ffb482', '#8de5a1', '#ff9f9b', 
                                     '#d0bbff', '#debb9b', '#fab0e4', '#cfcfcf',
                                     '#fffea3', '#b9f2f0', 
                                    '#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', 
                                     '#a6d854', '#ffd92f', '#e5c494', '#b3b3b3',
                                    '#f77189', '#bb9832', '#50b131', '#36ada4', '#3ba3ec', '#e866f4',
                                    
                                    '#f77189', '#bb9832', '#50b131', '#36ada4', '#3ba3ec', '#e866f4']

In [ ]:
sc.pl.umap(shuffle(P22_gaba_types), color=['Sample'],  )
sc.pl.umap(P22_gaba_types, color=['Type', 'Subclass', 'leiden'], legend_loc='on data', legend_fontsize=10)

In [ ]:
sc.pl.correlation_matrix(Pvalb, groupby='Type',show_correlation_numbers=True)

In [ ]:
plt.rcParams['font.size'] = 11
sc.pl.correlation_matrix(P22_gaba_types, groupby='Type',show_correlation_numbers=True)

In [ ]:
P22_gaba_types.write_h5ad('h5ads/P22Sham_gaba_v2.h5ad')

In [ ]:
rd = pd.DataFrame(P22_gaba_types.obsm['X_umap'], index=P22_gaba_types.obs_names, columns=['UMAP1', 'UMAP2'])

In [ ]:
P22_gaba_types.obs['cluster'] = P22_gaba_types.obs['Type'].cat.rename_categories(np.arange(len(P22_gaba_types.obs['Type'].values.categories)))
P22_gaba_types.obs['cluster_label'] = P22_gaba_types.obs['Type']

In [ ]:
cl = P22_gaba_types.obs['cluster']
cl_label = P22_gaba_types.obs['cluster_label']

In [ ]:
rd.to_csv('rd.csv')

In [ ]:
cl.to_csv('cl.csv')
cl_label.to_csv('cl_label.csv')

In [ ]:
rd[rd.isna()]

In [ ]:
rd.to_

In [ ]:
nx.graph.n

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import hypergeom
import networkx as nx
from sklearn.neighbors import kneighbors_graph


def get_knn_graph(rd_dat, cl, cl_df, k=15, knn_outlier_th=2, outlier_frac_th=0.5):
    knn_result = kneighbors_graph(rd_dat, n_neighbors=k, mode='connectivity').toarray()
    knn = knn_result.astype(int)
    knn_dist = knn_result * np.linalg.norm(rd_dat, axis=1)
    cl_knn_dist_mean = {x: knn_dist[cl == x][:, 1:].mean() for x in np.unique(cl)}
    cl_knn_dist_sd = {x: knn_dist[cl == x][:, 1:].std() for x in np.unique(cl)}
    cl_knn_dist_th = {x: mean + knn_outlier_th * sd for x, mean, sd in zip(np.unique(cl), cl_knn_dist_mean.values(), cl_knn_dist_sd.values())}
    
    knn_dist_th = [cl_knn_dist_th[cl[int(i)]] for i in np.arange(len(knn))]
    outlier = (knn_dist > np.array(knn_dist_th)[:, np.newaxis])
    select_cells = np.array(range(len(outlier)))[outlier.sum(axis=1) < outlier_frac_th]
    
    pred_result = predict_knn(knn[select_cells], np.array(range(len(rd_dat))), cl)
    pred_prob = pred_result['pred_prob']
    knn_cell_cl_counts = np.round(pred_prob * knn.shape[1])
    
    knn_cl_cl_counts = np.zeros((len(np.unique(cl)), len(np.unique(cl))))
    for i in np.unique(cl):
        x = np.where(cl == i)[0]
        knn_cl_cl_counts[i, :] = knn_cell_cl_counts[x, :].sum(axis=0)
    
    knn_cl_df = pd.DataFrame(knn_cl_cl_counts, columns=np.unique(cl))
    knn_cl_df['cl.from'] = np.arange(len(np.unique(cl)))
    knn_cl_df = knn_cl_df.melt(id_vars=['cl.from'], var_name='cl.to', value_name='Freq')
    
    from_size = knn_cl_cl_counts.sum(axis=1)
    to_size = knn_cl_cl_counts.sum(axis=0)
    total = knn_cl_cl_counts.sum()
    
    knn_cl_df['cl.from.total'] = from_size[knn_cl_df['cl.from']]
    knn_cl_df['cl.to.total'] = to_size[knn_cl_df['cl.to']]
    knn_cl_df = knn_cl_df[knn_cl_df['Freq'] > 0]
    knn_cl_df['pval.log'] = knn_cl_df['odds'] = 0
    
    for i, row in knn_cl_df.iterrows():
        q = row['Freq'] - 1
        k = row['cl.from.total']
        m = row['cl.to.total']
        n = total - m
        pval_log = hypergeom.logsf(q, M=n+m, n=n, N=k, loc=1)
        knn_cl_df.at[i, 'pval.log'] = pval_log
        odds = (q + 1) / (k * m / total)
        knn_cl_df.at[i, 'odds'] = odds
    
    knn_cl_df['frac'] = knn_cl_df['Freq'] / knn_cl_df['cl.from.total']
    knn_cl_df['cl.from.label'] = cl_df.loc[knn_cl_df['cl.from'], 'cluster_label'].values
    knn_cl_df['cl.to.label'] = cl_df.loc[knn_cl_df['cl.to'], 'cluster_label'].values
    
    return {
        'knn_result': knn_result,
        'pred_result': pred_result,
        'knn_cl_df': knn_cl_df
    }

def plot_constellation(knn_cl_df, cl_center_df, out_dir, node_label="cluster_id", exxageration=2, curved=True,
                       plot_parts=False, plot_hull=None, plot_height=25, plot_width=25, node_dodge=False,
                       label_size=5, max_size=10, label_repel=False):
    pass  # Your Python implementation of plot_constellation goes here



In [ ]:
get_knn_graph(rd_dat=rd, cl=cl, cl_df=cl_label)

# Final Type refinement

## QC metric

In [ ]:
P22_glut_types = sc.read_h5ad('h5ads_cmprsd/P22Sham_glut_v2.h5ad')
P22_gaba_types = sc.read_h5ad('h5ads_cmprsd/P22Sham_gaba_v2.h5ad')

In [ ]:
def clust_obs_plot(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    
    x,y = ser.index, ser.values
    
    plt.figure(figsize=(15,4))
    plt.bar(x,y)
    plt.axhline(np.median(adata.obs[obs_id]), color='r')
    plt.ylabel(obs_id)
    plt.xticks(rotation=90)
    plt.show()
    return x, y

In [ ]:
clust_obs_plot(P22_glut_types, 'P21 Mapping Prob', obs_id_split='Type')
clust_obs_plot(P22_glut_types, 'pct_counts_mt', obs_id_split='Type')
clust_obs_plot(P22_glut_types, 'Doublet Score', obs_id_split='Type')
clust_obs_plot(P22_glut_types, 'total_counts', obs_id_split='Type')

In [ ]:
clust_obs_plot(P22_gaba_types, 'P21 Mapping Prob', obs_id_split='Type')
clust_obs_plot(P22_gaba_types, 'pct_counts_mt', obs_id_split='Type')
clust_obs_plot(P22_gaba_types, 'Doublet Score', obs_id_split='Type')
clust_obs_plot(P22_gaba_types, 'total_counts', obs_id_split='Type')

## Mapping to V1 P21 multiome

### Gluta

In [ ]:
P21_V1 = rem_fem(sc.read_h5ad('P21full_glut_v3.h5ad'))
P21_V1 = P21_V1[P21_V1.obs['Study']!='2022 RNA',:].copy()
P21_V1.obs['Type'] = P21_V1.obs['Type_nn_dists']
P21_V1.X = P21_V1.raw.X

In [ ]:
P22_glut = sc.read_h5ad('h5ads/P22Sham_glut_v2.h5ad')
P22_glut.X = P22_glut.raw.X

**The change below to plot_conf_matx() fixes the indeing issue.**

In [ ]:
#code
#import
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scanpy as sc
import pandas as pd
from harmony import harmonize
from sklearn.metrics.cluster import adjusted_rand_score
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.utils import shuffle
from anndata import AnnData
from sklearn.cluster import KMeans
from typing import Union, Optional, Tuple, Collection, Sequence, Iterable
from scipy.stats import hypergeom
import sklearn.preprocessing
import seaborn as sn
from random import sample
import pickle
#from matplotlib_venn import venn3, venn3_circles
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples


from matplotlib import gridspec
from sklearn.metrics import confusion_matrix, adjusted_rand_score, roc_curve, auc, classification_report, f1_score, cohen_kappa_score
import plotly.graph_objects as go
from itertools import cycle, islice
from sklearn.preprocessing import label_binarize

import scanpy as sc
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.set_figure_params(dpi=100, dpi_save=200)


import scrublet as scr

from typing import Optional

# import warnings
# warnings.filterwarnings("ignore")

import matplotlib as mpl
from matplotlib import gridspec
import xgboost as xgb
from sklearn.metrics import accuracy_score


sc.settings.verbosity = 0           # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.set_figure_params(dpi=75, dpi_save=200)

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

def test_func():
    print('test')
    
def test2():
    print('test2')
def rem_fem(adata):
    for j in list(adata.obs.Type_nn_dists.values.categories):
        if ('Fem' in j):
            adata = adata[adata.obs['Type_nn_dists']!=j,:]
            adata = adata[adata.obs['Type_leiden']!=j,:]
            adata = adata[adata.obs['Type']!=j,:]
    return adata
def pipeline_short(adata, batch_correct, batch_ID):
                   
    if sp.sparse.issparse(adata.X):
        if np.any(adata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(adata.X<0):
            raise Exception("Matrix contains negative values")

    
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch_ID) #HVGs
    adata.raw = adata
    sc.pp.scale(adata, max_value=10) #scale
    sc.tl.pca(adata, svd_solver='arpack') #run PCA
    
    if (batch_correct):
        Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = batch_ID)
        adata.obsm['X_harmony'] = Z
        sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_harmony', n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

    else:
        sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)
        
def pre_process(adata, batch_ID):
    
    if sp.sparse.issparse(adata.X):
        if np.any(adata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(adata.X<0):
            raise Exception("Matrix contains negative values")
    
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch_ID) #HVGs
    adata.raw = adata
    sc.pp.scale(adata, max_value=10) #scale
    sc.tl.pca(adata, svd_solver='arpack') #run PCA
    

def pipeline(adata, 
             batch_correct: Optional[bool] = None,
             batch_ID: Optional[str] = None):
    
    if (batch_correct):
        Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = batch_ID)
        adata.obsm['X_harmony'] = Z
        sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_harmony', n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

    else:
        sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

def clust_obs_plot(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    
    x,y = ser.index, ser.values
    
    plt.figure(figsize=(15,4))
    plt.bar(x,y)
    plt.axhline(np.median(adata.obs[obs_id]), color='r')
    plt.ylabel(obs_id)
    
    return x, y

def DE(adata, obs_id, obs_id_test, ref, pts_thresh, lf_thresh):

    sc.tl.rank_genes_groups(adata, groupby=obs_id, groups=[obs_id_test], 
                                reference=ref, method='t-test', pts=True, use_raw=True)

    lfcs = adata.uns['rank_genes_groups']['logfoldchanges'].astype([(obs_id_test, '<f8')]).view('<f8') 

    l231_genes = adata.uns['rank_genes_groups']['pts']

    lfcs = []
    p_adj = []
    names = list(adata.uns['rank_genes_groups']['names'].astype([(obs_id_test, '<U50')]).view('<U50'))
    logfoldchanges = adata.uns['rank_genes_groups']['logfoldchanges'].astype([(obs_id_test, '<f8')]).view('<f8')
    pvals_adj = adata.uns['rank_genes_groups']['pvals_adj'].astype([(obs_id_test, '<f8')]).view('<f8')

    for i in l231_genes.index:
        lfcs.append(logfoldchanges[names.index(i)])
        p_adj.append(pvals_adj[names.index(i)])

    l231_genes['LF'] = lfcs
    l231_genes['p_adj'] = p_adj
    
    #plt.hist(l231_genes[obs_id_test].values)

    l231_genes = l231_genes[l231_genes[obs_id_test]>pts_thresh]
    
    sort_LF = l231_genes.sort_values('LF', ascending=False)
    
    a = np.where(sort_LF[sort_LF['LF']>0.6]['p_adj'].values<0.05)[0].shape[0]
    b = sort_LF[sort_LF['LF']>0.6].shape[0]
    if(a == b):
        print('cutoffs are good at 1.5 FC level')
    else:
        print(a,b)
        
    return sort_LF[sort_LF['LF']>lf_thresh]
        
def clust_obs(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    return ser.index, ser.values

def res_tune(adata, def_labels):
    
    adata_ari = []
    adata_num_clusts = []
    for res in np.arange(1,2.1, 0.1):
        sc.tl.leiden(adata, resolution=res)

        adata_num_clusts.append(len(adata.obs.leiden.values.categories))
        adata_ari.append(adjusted_rand_score(def_labels, list(adata.obs.leiden.values)))
        
    return adata_num_clusts, adata_ari

from math import log, e
def entropy2(labels, base=None):
  """ Computes entropy of label distribution. """

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value,counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  # Compute entropy
  base = e if base is None else base
  for i in probs:
    ent -= i * log(i, base)

  return ent

def check_clust(adata, clus, ref_age):
    sc.pl.umap(adata[adata.obs.leiden==clus], color=[ref_age+'-2022 Mapping Prob', ref_age+'-2022 Mapping Label'])
    print(adata[adata.obs.leiden==clus].shape, 'mean prob', np.mean(adata[adata.obs.leiden==clus].obs[ref_age+'-2022 Mapping Prob']))
    ser = adata[adata.obs.leiden==clus].obs['Sample'].value_counts()
    plt.bar(ser.index, ser.values,)
    plt.xticks(rotation=90)
    plt.show()
    ser = adata[adata.obs.leiden==clus].obs[ref_age+'-2022 Mapping Label'].value_counts()
    plt.bar(ser.index, ser.values,)
    plt.xticks(rotation=90)
    plt.show()
    
def maj_vote_annot(adata, clust_id, putative_annot, final_annot, vlmcendo=False):
    clust_list = []
    for i in adata.obs[clust_id].values.categories:
        clust = adata[adata.obs[clust_id]==i,:]
        clust_df = clust.obs[putative_annot].value_counts()
        if (clust_df.index[0]=='Unassigned'):
            biggest_cat = clust_df.index[1]
        else: biggest_cat = clust_df.index[0]
        if (vlmcendo):
            if (biggest_cat in ('VLMC', 'Endo')):
                clust.obs[putative_annot+'_maj'] = ['VLMC+Endo']*clust.shape[0]
                clust_list.append(clust)
            else:
                clust.obs[putative_annot+'_maj'] = [biggest_cat]*clust.shape[0]
                clust_list.append(clust)
        else:
            clust.obs[putative_annot+'_maj'] = [biggest_cat]*clust.shape[0]
            clust_list.append(clust)

    adata_annot = clust_list[0].concatenate(clust_list[1:], index_unique=None)
    adata_annot.obs[final_annot] = adata_annot.obs[putative_annot+'_maj']
    a = sc.pl.dotplot(adata_annot, 'Mdga1', 'leiden',) #to fix categories
    return adata_annot

def clus_sample_bars(adata, a_, b_, samp_id, clus_id, size=(13,9.5)):
    adata_ser = adata.obs[samp_id].value_counts(normalize=True)
    plt.bar(adata_ser.index, adata_ser.values)
    plt.xticks(rotation=90)
    plt.title('Overall Dataset')
    plt.show()
    a,b = 0 ,0 
    fig, axs = plt.subplots(a_,b_, figsize=size)

    for i in adata.obs[clus_id].values.categories:
        clus = adata[adata.obs[clus_id]==i,:]
        clus_ser = clus.obs[samp_id].value_counts(normalize=True)
        axs[a,b].bar(clus_ser.index, clus_ser.values)
        axs[a,b].set_title(i+' ('+str(clus.shape[0])+' cells)')
        axs[a,b].set_ylabel('Fraction')
        axs[a,b].set_xticklabels(clus_ser.index, rotation=90)
        b = b + 1
        if (b>=b_):
            b = 0
            a = a + 1
    plt.tight_layout()
    
    
def euclidean_distance(vector1, vectors_list):
    # Ensure that the input vectors are numpy arrays
    vector1 = np.array(vector1)
    vectors_list = [np.array(vector) for vector in vectors_list]
    
    # Calculate the Euclidean distance between the input vector and each vector in the list
    distances = [np.linalg.norm(vector1 - vector) for vector in vectors_list]
    return distances

def nn_voting(adata, type_old, type_new, delta_thresh):
    adata.obs['idx'] = np.arange(adata.shape[0])

#     #re-make graph using UMAP space for voting below
#     sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_umap')
#     sc.tl.umap(adata)

    adata.obs[type_new] = adata.obs[type_old]
    adata_un_idx = adata[adata.obs[type_old]=='Unassigned',:].obs.idx.values #indices of unassigned cells
    neigh_matx = adata.obsp['distances'].A #each row's nonzero entries tells neighbors
    adata_iGBs = list(adata.obs[type_old].values) #all the step1 assignments
    n_neighbs = adata.uns['neighbors']['params']['n_neighbors']
    print(adata.uns['neighbors']['params'])
    print(' ')
    
    print("Pre-voting unassigned: ", adata_iGBs.count('Unassigned'),
          adata_iGBs.count('Unassigned')/adata.shape[0])
    
    
    delta = 1 #init delta
    while (delta>delta_thresh):    
        un_frac1 = adata_iGBs.count('Unassigned')/adata.shape[0] #frac of unassigned cells before voting

        #loop thru each cell
        for i in adata_un_idx:

            #so that it only loops thru still-unassigned cells after first pass
            if (adata.obs[type_new][i]=='Unassigned'):
                neighbs_idx = np.where(neigh_matx[i,:]>0)[0] #cell i's neighbors
                neighbs_iGBs = adata[neighbs_idx].obs[type_new] #the neighbors' iGBs

#                 #if there's a type in the neighbors that is majority, assign it
#                 if (neighbs_iGBs.value_counts()[0] > n_neighbs/2):
#                     adata_iGBs[i] = neighbs_iGBs.value_counts().index[0]
                
                #no need to be majority, just be biggest one
                adata_iGBs[i] = neighbs_iGBs.value_counts().index[0]
                #update the IDs to help assignment of next cell
                adata.obs[type_new] =  pd.Categorical(adata_iGBs)

        un_frac2 = adata_iGBs.count('Unassigned')/adata.shape[0] #frac of unassigned cells after voting

        delta = (un_frac1-un_frac2)/un_frac1   #stop when this changes by less than 1%
        print(delta, un_frac2)

    print("Post-voting unassigned: ", adata_iGBs.count('Unassigned'),
          adata_iGBs.count('Unassigned')/adata.shape[0])
    
    return adata


def make_hmap(adata,thres, y_obs='Type', x_obs='leiden', ):
    crosstab_data = pd.crosstab(adata.obs[y_obs], adata.obs[x_obs], 
                           normalize='index')
    # For rows
    row_order = np.argsort(-crosstab_data.values.max(axis=1))

    # For columns
    column_order = np.argsort(-crosstab_data.values.max(axis=0))

    reordered_rows = crosstab_data.index[row_order]
    reordered_crosstab_rows = crosstab_data.iloc[row_order]

    # For columns
    reordered_columns = crosstab_data.columns[column_order]
    reordered_crosstab = reordered_crosstab_rows[reordered_columns]
    #return reordered_crosstab
    
    
    from sklearn.metrics import adjusted_rand_score

    

    labels_true = adata.obs[y_obs]
    labels_pred = adata.obs[x_obs]

    # Compute the Adjusted Rand Index
    ari = adjusted_rand_score(labels_true, labels_pred)
    
    return [reordered_crosstab.applymap(lambda value: value if value > thres else np.round(value, 2)),
           ari]
def freq_scatter(x, y, x_lab, y_lab, hue_, unity_lim, low_lim):

    if (x.shape[0]!=y.shape[0]): 
        print('x and y are different by', np.abs(x.shape[0]-y.shape[0]), 'categories')
    
    if (y.shape[0]>x.shape[0]):
        y = y[x.index]
    else:
        x = x[y.index]

    df = pd.DataFrame(index=list(x.index.values), columns=[x_lab, y_lab],
                      data=np.transpose(np.array([x.values, y.values])))

    df[hue_] = df.index
    sn.scatterplot(data=df, x=x_lab, y = y_lab, hue=hue_, s=50)
    plt.legend(bbox_to_anchor=(1.01, 1.03), loc='upper left', fontsize=14, ncol=2)
    plt.plot(np.linspace(low_lim,unity_lim), np.linspace(low_lim,unity_lim), ls='--', 
             color='black', linewidth=0.75)
    plt.title('Pearson R: '+str(np.round(sp.stats.pearsonr(x, y)[0], 3)))
    plt.grid(False)
    #plt.loglog()
    


from anndata import AnnData
from typing import Union, Optional, Tuple, Collection, Sequence, Iterable

def module_score(adata:AnnData, genes_use: list, score_name: Optional[str] = None, verbose: bool = True):
    
    """\
    Compute module scores for all cells in adata as described in methods of RGC-dev paper.
    
    
    Parameters
    ----------
    adata
        The (annotated) data matrix of shape `n_obs` × `n_vars`.
        Rows correspond to cells and columns to genes.
    genes_use
        list of genes in module of interest
    score_name
        Name endowed to the module score to be computed
        e.g. "Mod1"
    verbose
        Inform user of fraction of module genes that are in adata
        
    Returns
    -------
    adata with a new .obs called score_name
    
    """
    
    if (score_name==None):
        score_name = str(input("Provide a name for this score (no spaces): "))
        
    genes_use0 = genes_use
    genes_use = list(set(genes_use).intersection(adata.var_names))#genes that are both in module and `adata`
    
     
    if (len(genes_use) == 0):
        raise ValueError("Error : Must provide a list of genes that are present in the data")
        
    
    if (verbose):
        if(len(genes_use0) > len(genes_use)):
            n = len(genes_use0) - len(genes_use)
            print(score_name,": Note that", n, "of the", len(genes_use0), "genes in your module do not exist in the data set." )
        else:
            print(score_name,": Note that all of the", len(genes_use), "genes in your module are in the data set." )
    
    
    
    adata_score = adata.copy()
    adata_score = adata[:,genes_use]
    
    counts_modgenes = adata_score.X.toarray() #all cells, module genes
    counts_all = adata.X.toarray() #all cells, all genes
    #scores = np.mean(counts_modgenes, axis=1) - np.mean(counts_all, axis=1) #(row means of counts_modgenes ) - (row means of counts_all)
    scores = np.mean(counts_modgenes, axis=1) #(row means of counts_modgenes )

    adata.obs[score_name] = scores
    
    return genes_use    
    

def clean_subclasses(adata, rep):
    #define avg PC position of each type. Gotta use a type with no "Unassigned" group
    typ_mean_dict = {}
    for i in adata.obs['Subclass'].values.categories:
        typ_mean_dict[i] = np.mean(adata[adata.obs['Subclass']==i,:].obsm[rep][:,0:40], axis=0)

    #assign based on proximity to avg type
    dists_list = []
    dict_keys = list(typ_mean_dict.keys())
    for i in range(adata.shape[0]):
        typ_ = adata.obs['Subclass'][i]
        dists = euclidean_distance(adata.obsm[rep][i,0:40], list(typ_mean_dict.values()))

        #dist = dists[dict_keys.index(typ_)]
        typ = dict_keys[np.argmin(dists)]
        #types.append(typ)
        if (typ!=typ_):
            dists_list.append(100)
        else: dists_list.append(0)

    adata.obs['Dist to Subclass'] = dists_list
    
def make_dict(adata, obs_id):

    adata_dict = {}
    for num,i in enumerate(adata.obs[obs_id].values.categories):
        adata_dict[i] = num
        
    #adata_dict['Unassigned'] = num + 1
    
    return adata_dict
    

def plotConfusionMatrix(
    ytrue,
    ypred,
    type,
    xaxislabel,
    yaxislabel,
    title,
    train_dict,
    test_dict=None,
    re_order=None,
    re_order_cols = None,
    re_index = None,
    re_order_rows = None,
    save_as=None,):
    
    #very bad
    numbertrainclasses = len(set(ypred))
    numbertestclasses = len(set(ytrue))
    
    #cfm is 11x11 b/c 11 is = y_true U y_pred
    confusionmatrix = confusion_matrix(y_true = ytrue, y_pred = ypred)
        
    #only need this when mapping b/c if validaiton, all classes will be used and cfm will be constructed properly
    if type == 'mapping':
        rows = np.where(np.sum(confusionmatrix, axis=1)>0)[0]
        cols = np.where(np.sum(confusionmatrix, axis=0)>0)[0]
        
        cfm = confusionmatrix[rows,:][:,cols]
        
        #show all columns, even ones with no mapping
        
        #this was changed 10/12/2023 to address unassigned issue
        cfm_z = np.zeros((len(test_dict),len(train_dict)))
        #cfm_z = np.zeros((len(rows),len(cols)))

        cfm_z[:, np.array(pd.Categorical(ypred).categories, dtype='int')]=cfm 
        confusionmatrix = cfm_z
        
        #always keep only as many as rows as num of test classes
        #but, b/c of python's 0 indexing, if the number of training classes is in the 
        #y_pred list, then that means there was an unassigned
#       if numbertrainclasses in ypred:
#         confusionmatrix = confusionmatrix[0:numbertestclasses,0:numbertrainclasses+1]#for Unassigned
#       else:
#         confusionmatrix = confusionmatrix[0:numbertestclasses,0:numbertrainclasses]
    
        confmatpercent = confusionmatrix/np.sum(confusionmatrix, axis=1).reshape(-1,1)

        conf_df = pd.DataFrame(confmatpercent)
        conf_df.index = list(test_dict.keys())

        #name columns of conf mat
        if(len(conf_df.columns)>len(train_dict)):
            conf_df.columns = list(train_dict.keys())+['Unassigned']
        else:
            conf_df.columns = list(train_dict.keys())


        if (re_order):
            conf_df = conf_df[re_order_cols]
        if (re_index):
            conf_df = conf_df.reindex(re_order_rows)

        diagcm = conf_df.to_numpy()
    
    
        xticksactual = list(conf_df.columns)
        
        #print(conf_df)
        
    
    else:
        confmatpercent = np.zeros(confusionmatrix.shape)
        for i in range(confusionmatrix.shape[0]):
            if np.sum(confusionmatrix[i,:]) != 0:
                confmatpercent[i,:] = confusionmatrix[i,:]/np.sum(confusionmatrix[i,:])
            else:
                confmatpercent[i,:] = confusionmatrix[i,:]
            diagcm = confmatpercent
            xticks = np.linspace(0, confmatpercent.shape[1]-1, confmatpercent.shape[1], dtype = int)
        xticksactual = []
        for i in xticks:
            if i != numbertrainclasses:
                xticksactual.append(list(train_dict.keys())[i])
            else:
                xticksactual.append('Unassigned')
        
    dot_max = np.max(diagcm.flatten())
    dot_min = 0
    if dot_min != 0 or dot_max != 1:
        frac = np.clip(diagcm, dot_min, dot_max)
        old_range = dot_max - dot_min
        frac = (frac - dot_min) / old_range
    else:
        frac = diagcm
    xvalues = []
    yvalues = []
    sizes = []
    for i in range(diagcm.shape[0]):
        for j in range(diagcm.shape[1]):
            xvalues.append(j)
            yvalues.append(i)
            sizes.append((frac[i,j]*35)**1.5)
    size_legend_width = 0.5
    height = diagcm.shape[0] * 0.3 + 1
    height = max([1.5, height])
    heatmap_width = diagcm.shape[1] * 0.35
    width = (
        heatmap_width
        + size_legend_width
        )
    fig = plt.figure(figsize=(width, height))
    axs = gridspec.GridSpec(
        nrows=2,
        ncols=2,
        wspace=0.02,
        hspace=0.04,
        width_ratios=[
                    heatmap_width,
                    size_legend_width
                    ],
        height_ratios = [0.5, 10]
        )
    dot_ax = fig.add_subplot(axs[1, 0])
    dot_ax.scatter(xvalues,yvalues, s = sizes, c = 'blue', norm=None, edgecolor='none')
    y_ticks = range(diagcm.shape[0])
    dot_ax.set_yticks(y_ticks)
    if type == 'validation':
        dot_ax.set_yticklabels(list(train_dict.keys()))
    elif type == 'mapping':
      #dot_ax.set_yticklabels(list(test_dict.keys()))
        dot_ax.set_yticklabels(list(conf_df.index))
    x_ticks = range(diagcm.shape[1])
    dot_ax.set_xticks(x_ticks)
    dot_ax.set_xticklabels(xticksactual, rotation=90)
    dot_ax.tick_params(axis='both', labelsize='small')
    dot_ax.grid(True, linewidth = 0.2)
    dot_ax.set_axisbelow(True)
    dot_ax.set_xlim(-0.5, diagcm.shape[1] + 0.5)
    ymin, ymax = dot_ax.get_ylim()
    dot_ax.set_ylim(ymax + 0.5, ymin - 0.5)
    dot_ax.set_xlim(-1, diagcm.shape[1])
    dot_ax.set_xlabel(xaxislabel)
    dot_ax.set_ylabel(yaxislabel)
    dot_ax.set_title(title)
    size_legend_height = min(1.75, height)
    wspace = 10.5 / width
    axs3 = gridspec.GridSpecFromSubplotSpec(
        2,
        1,
        subplot_spec=axs[1, 1],
        wspace=wspace,
        height_ratios=[
                    size_legend_height / height,
                    (height - size_legend_height) / height
                    ]
        )
    diff = dot_max - dot_min
    if 0.3 < diff <= 0.6:
        step = 0.1
    elif diff <= 0.3:
        step = 0.05
    else:
        step = 0.2
    fracs_legends = np.arange(dot_max, dot_min, step * -1)[::-1]
    if dot_min != 0 or dot_max != 1:
        fracs_values = (fracs_legends - dot_min) / old_range
    else:
        fracs_values = fracs_legends
    size = (fracs_values * 35) ** 1.5
    size_legend = fig.add_subplot(axs3[0])
    size_legend.scatter(np.repeat(0, len(size)), range(len(size)), s=size, c = 'blue')
    size_legend.set_yticks(range(len(size)))
    labels = ["{:.0%}".format(x) for x in fracs_legends]
    if dot_max < 1:
        labels[-1] = ">" + labels[-1]
    size_legend.set_yticklabels(labels)
    size_legend.set_yticklabels(["{:.0%}".format(x) for x in fracs_legends])
    size_legend.tick_params(axis='y', left=False, labelleft=False, labelright=True)
    size_legend.tick_params(axis='x', bottom=False, labelbottom=False)
    size_legend.spines['right'].set_visible(False)
    size_legend.spines['top'].set_visible(False)
    size_legend.spines['left'].set_visible(False)
    size_legend.spines['bottom'].set_visible(False)
    size_legend.grid(False)
    ymin, ymax = size_legend.get_ylim()
    size_legend.set_ylim(ymin, ymax + 0.5)
    
    if (save_as is not None):
        fig.savefig(save_as, bbox_inches = 'tight')

    return diagcm, xticksactual, axs



#This helper method plots validation plots in sequential order (i.e. first plot is for first batch, second plot is for second batch, etc.)
def plot_validation_plots(validation_label_train_70, valid_predlabels_train_70, train_dict, save_as=None):
    
    ARI = adjusted_rand_score(labels_true = validation_label_train_70, 
                              labels_pred = valid_predlabels_train_70)
    
    
    c = 0
    for i in range(validation_label_train_70.shape[0]):
        if (validation_label_train_70[i]!=valid_predlabels_train_70[i]):
            c = c +1
    acc = (1 - c/len(validation_label_train_70))*100
    
           
    validationconfmat, validationxticks, validationplot = plotConfusionMatrix(
    ytrue = validation_label_train_70,
    ypred = valid_predlabels_train_70,
    train_dict=train_dict,
    type = 'validation',
    save_as = save_as,
    title = 'ARI = {:.3f}, Accuracy = {:.3f}'.format(ARI, acc),
    xaxislabel = 'Predicted',
    yaxislabel = 'True'
    )

def plot_mapping(test_labels, test_predlabels, test_dict, train_dict, 
                 xaxislabel, yaxislabel,
                re_order=None,
    re_order_cols = None,
                 re_index = None,
    re_order_rows = None, save_as=None):
    
    ARI = adjusted_rand_score(labels_true = test_labels, 
                              labels_pred = test_predlabels)
    NCE = calculateNCE(labels_true = test_labels, labels_pred = test_predlabels)
    
 
    
           
    mappingconfmat, mappingxticks, mappingplot = plotConfusionMatrix(
    ytrue = test_labels,
    ypred = test_predlabels,
    test_dict=test_dict,
    train_dict=train_dict,
    type = 'mapping',
    save_as = save_as,
    title = 'ARI = {:.3f}, NCE = {:.3f}'.format(ARI, NCE),
    xaxislabel =xaxislabel,
    yaxislabel = yaxislabel,
        re_order=re_order,
    re_order_cols = re_order_cols,
        re_index = re_index,
    re_order_rows = re_order_rows,
    ) 
    return mappingconfmat, mappingxticks, mappingplot
      
      
#This helper method uses xgboost to train classifiers.
def trainclassifier(train_anndata, common_top_genes, obs_id, train_dict, eta,
                    max_cells_per_ident, train_frac, min_cells_per_ident):
    
    if sp.sparse.issparse(train_anndata.X):
        if np.any(train_anndata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(train_anndata.X<0):
            raise Exception("Matrix contains negative values")

    
    start_time = time.time()
    
    numbertrainclasses = len(train_anndata.obs[obs_id].values.categories)

    xgb_params_train = {
            'objective':'multi:softprob',
            'eval_metric':'mlogloss',
            'num_class':numbertrainclasses,
            'eta':eta,
            'max_depth':4,
            'subsample': 0.6}
    nround = 200
    #Train XGBoost on 70% of training data and validate on the remaining data


    training_set_train_70 = []
    validation_set_train_70 = []
    training_label_train_70 = []
    validation_label_train_70 = []

    #loop thru classes to split for training and validation
    for i in train_anndata.obs[obs_id].values.categories:
        
        #how many cells in a class
        cells_in_clust = train_anndata[train_anndata.obs[obs_id]==i,:].obs_names #cell names
        n = min(max_cells_per_ident,round(len(cells_in_clust)*train_frac))
        
        #sample 70% for training and rest for validation
        train_temp = np.random.choice(cells_in_clust,n,replace = False)
        validation_temp = np.setdiff1d(cells_in_clust, train_temp)
        
        #upsample small clusters
        if len(train_temp) < min_cells_per_ident:
            train_temp_bootstrap = np.random.choice(train_temp, size = min_cells_per_ident - int(len(train_temp)))
            train_temp = np.hstack([train_temp_bootstrap, train_temp])
        
        #store training and validation **names** of cells in vectors, which update for every class
        training_set_train_70 = np.hstack([training_set_train_70,train_temp])
        validation_set_train_70 = np.hstack([validation_set_train_70,validation_temp])
        
        #store training and validation **labels** of cells in vectors, which update for every class
        training_label_train_70 = np.hstack([training_label_train_70,np.repeat(train_dict[i],len(train_temp))])
        validation_label_train_70 = np.hstack([validation_label_train_70,np.repeat(train_dict[i],len(validation_temp))])

        #need train_dict b/c XGboost needs number as class labels, not words
        #this is only deconvulted later in plotting function
        
    #put data in XGB format
    X_train = train_anndata[training_set_train_70,common_top_genes].X
    train_matrix_train_70 = xgb.DMatrix(data = X_train, label = training_label_train_70, 
                                        feature_names = common_top_genes)
    
    X_valid = train_anndata[validation_set_train_70,common_top_genes].X
    validation_matrix_train_70 = xgb.DMatrix(data = X_valid, label = validation_label_train_70, 
                                             feature_names = common_top_genes)

    del training_set_train_70, validation_set_train_70, training_label_train_70
    
    #Train on 70%
    bst_model_train_70 = xgb.train(
        params = xgb_params_train,
        dtrain = train_matrix_train_70,
        num_boost_round = nround)
    
    #Validate on 30%
    #a validation_cells x numclasses matrix, with each vector containing prob association with the classes
    validation_pred_train_70 = bst_model_train_70.predict(data = validation_matrix_train_70)
    
    #for each cell, go through vec of probs and take index of max prob: that's assignment
    valid_predlabels_train_70 = np.zeros((validation_pred_train_70.shape[0]))
    for i in range(validation_pred_train_70.shape[0]):
        valid_predlabels_train_70[i] = np.argmax(validation_pred_train_70[i,:])
        
    
    #Train on 100%
    #Train XGBoost on the full training data
    training_set_train_full = []
    training_label_train_full = []

    for i in train_anndata.obs[obs_id].values.categories.values:
        train_temp = train_anndata.obs.index[train_anndata.obs[obs_id].values == i]
        if len(train_temp) < 100:
            train_temp_bootstrap = np.random.choice(train_temp, size = 100 - int(len(train_temp)))
            train_temp = np.hstack([train_temp_bootstrap, train_temp])
        
        #indices of cells in class
        training_set_train_full = np.hstack([training_set_train_full,train_temp])
        
        #labels of cells in class: [label*N_class] stacked onto previous classes
        training_label_train_full = np.hstack([training_label_train_full,np.repeat(train_dict[i],len(train_temp))])


    X_train_full = train_anndata[training_set_train_full,common_top_genes].X
    full_training_data = xgb.DMatrix(data = X_train_full, label = training_label_train_full, 
                                     feature_names = common_top_genes)

    del training_set_train_full, training_label_train_full

    bst_model_full_train = xgb.train(
        params = xgb_params_train,
        dtrain = full_training_data,
        num_boost_round = nround)

    
    
    print('trainclassifier() complete after', np.round(time.time() - start_time), 'seconds')
    
    
    f1 = f1_score(validation_label_train_70, valid_predlabels_train_70, average = None)

    
    #real labels of validation set, predicted labels, classifier.
    #recall these are all integers that are deconvulted later in plotting using the dicts
    return validation_label_train_70, valid_predlabels_train_70, bst_model_full_train, f1


#This helper method predicts the testing cluster labels.
def predict(train_anndata, common_top_genes, bst_model_train_full, test_anndata, 
            train_obs_id, test_dict, test_obs_id):
    
    
    if sp.sparse.issparse(train_anndata.X):
        if np.any(train_anndata.X.A<0):
            raise Exception("Training matrix contains negative values")
    else: 
        if np.any(train_anndata.X<0):
            raise Exception("Training matrix contains negative values")

    if sp.sparse.issparse(test_anndata.X):
        if np.any(test_anndata.X.A<0):
            raise Exception("Testing matrix contains negative values")
    else: 
        if np.any(test_anndata.X<0):
            raise Exception("Testing matrix contains negative values")
  
    
    #Predict the testing cluster labels
    #how many classes mapping to 
    numbertrainclasses = len(train_anndata.obs[train_obs_id].values.categories)
    
    #put testing data into XGB format
    full_testing_data = xgb.DMatrix(data = test_anndata[:,common_top_genes].X, 
                                    feature_names=common_top_genes)
    
    #a testing_cells x numclasses matrix, with each vector containing prob association with the classes
    test_prediction = bst_model_train_full.predict(data = full_testing_data)

    #for each cell, go through vec of probs and take index of max prob (if greater than ...): that's assignment

    
    test_predlabels = np.zeros((test_prediction.shape[0]))
    for i in range(test_prediction.shape[0]):
        if np.max(test_prediction[i, :]) > 1.1*(1/numbertrainclasses):
            test_predlabels[i] = np.argmax(test_prediction[i,:])
        
        #"unassigned" is a label one larger than all b/c python begins indexing at 0
        else:
            test_predlabels[i] = numbertrainclasses
        
    test_labels = np.zeros(len(test_anndata.obs[test_obs_id].values))
    for i,l in enumerate(test_anndata.obs[test_obs_id].values):
        test_labels[i] = test_dict[l]

    #actual labels of testing set, the labels that test set mapped to 
    return test_labels, test_predlabels, test_prediction

def calculateNCE(labels_true,labels_pred):
    X = labels_true
    Y = labels_pred
    contTable = confusion_matrix(X,Y)[0:len(np.unique(X)), 0:len(np.unique(Y))]
    a = np.sum(contTable, axis = 1)
    b = np.sum(contTable, axis = 0)
    N = np.sum(contTable)
    pij = contTable/N
    pi = a/N
    pj = b/N
    Hyx = np.zeros(contTable.shape)
    for i in range(contTable.shape[0]):
        for j in range(contTable.shape[1]):
          if pij[i,j] == 0:
            Hyx[i,j] = 0
          else:
            Hyx[i,j] = pij[i,j]*np.log10(pij[i,j]/pi[i])
    CE = -np.sum(Hyx)
    Hyi = np.zeros(contTable.shape[1])
    for j in range(contTable.shape[1]):
      if pj[j] == 0:
       Hyi[j] = 0
      else:
        Hyi[j] = pj[j]*np.log10(pj[j])
    Hy = -np.sum(Hyi)
    NCE = CE/Hy
    return NCE

def train_validate(adata, adata_cell,preproc=False):
    if (preproc):
        adata_cell.X = adata_cell.raw.X
        sc.pp.highly_variable_genes(adata_cell, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs

    common_hvgs = list(set(adata[:,adata.var.highly_variable].var_names).intersection(set(adata_cell.var_names)))
    adata_m_dict = make_dict(adata, obs_id='leiden')
    adata_cell_dict = make_dict(adata_cell, obs_id='Subclass')
    print(len(common_hvgs), 'Shared HVGs')

    valid_truelabel_adata_cell, valid_predlabels_adata_cell, model_atlas_adata_cell = trainclassifier(train_anndata=adata_cell, 
                                                                                                 common_top_genes=common_hvgs, 
                                                                                                 obs_id='Subclass', 
                                                                                                 train_dict=adata_cell_dict, 
                                                                                                 eta=0.2,
                                                                                                 max_cells_per_ident=1000, 
                                                                                          train_frac=0.7, 
                                                                                                 min_cells_per_ident=100)

    plot_validation_plots(valid_truelabel_adata_cell, valid_predlabels_adata_cell, train_dict=adata_cell_dict)
    
    return model_atlas_adata_cell

def pairwise_map(adata_t0, adata_t1, test_lab, train_lab, t0_dict, t1_dict, recomp_HVGs, min_cells,
                x_lab, y_lab, union_hvgs):
    adata_t0 = adata_t0.copy() 
    adata_t1 = adata_t1.copy() 
    adata_t0.X = adata_t0.raw.X
    adata_t1.X = adata_t1.raw.X
    
    if (recomp_HVGs):
        sc.pp.highly_variable_genes(adata_t0, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs
        sc.pp.highly_variable_genes(adata_t1, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs
        t0_hvgs = list(adata_t0[:, adata_t0.var.highly_variable].var_names)
        t1_hvgs = list(adata_t1[:, adata_t1.var.highly_variable].var_names)
        
        if (union_hvgs):
            t0_t1hvgs_ = list(set(t0_hvgs).union(t1_hvgs))
            data_inter = list(set(adata_t1.var_names).intersection(set(adata_t0.var_names)))
            t0_t1hvgs = list(set(data_inter).intersection(set(t0_t1hvgs_)))

        else:
            t0_t1hvgs = list(set(t0_hvgs).intersection(t1_hvgs))

    
    else:
        t0_hvgs = list(adata_t0[:, adata_t0.var.highly_variable].var_names)
        t1_hvgs = list(adata_t1[:, adata_t1.var.highly_variable].var_names)
        t0_t1hvgs = list(set(t0_hvgs).intersection(t1_hvgs))

    print(len(t0_t1hvgs), 'shared HVGs')
    
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1,model_f1 = trainclassifier(train_anndata=adata_t1, 
                                                                                                 common_top_genes=t0_t1hvgs, 
                                                                                                 obs_id=train_lab, 
                                                                                                 train_dict=t1_dict, 
                                                                                                 eta=0.2,
                                                                                                 max_cells_per_ident=1000, 
                                                                                          train_frac=0.7, 
                                                                                                min_cells_per_ident=min_cells)

    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=t1_dict)

    test_labelst0vst1, test_predlabelst0vst1, test_prediction_t0vst1 = predict(train_anndata=adata_t1, 
                                                                     common_top_genes=model_t1.feature_names, 
                                                                     bst_model_train_full=model_t1, 
                                                                     test_anndata=adata_t0,
                                                                     train_obs_id=train_lab, 
                                                                     test_dict=t0_dict, 
                                                                     test_obs_id=test_lab)

    mappingconfmatt0vst1, mappingxtickst0vst1, mappingplott0vst1 = plot_mapping(test_labels=test_labelst0vst1, 
                 test_predlabels=test_predlabelst0vst1, 
                 test_dict=t0_dict, 
                 train_dict=t1_dict,
                     xaxislabel=x_lab, yaxislabel=y_lab,)
    
    test_items = test_labelst0vst1, test_predlabelst0vst1, test_prediction_t0vst1
    mapping_items = mappingconfmatt0vst1, mappingxtickst0vst1, mappingplott0vst1
    
    return test_items, mapping_items

def make_colors(PX_dict):
    PX_colors = []
    for i in PX_dict:
        for j in colors_subclass:
            if (j in i):
                PX_colors.append(colors_subclass[j])
    return PX_colors

def rem_fem(adata):
    for j in list(adata.obs.Type_nn_dists.values.categories):
        if ('Fem' in j):
            adata = adata[adata.obs['Type_nn_dists']!=j,:]
            adata = adata[adata.obs['Type_leiden']!=j,:]
    return adata

def list2dict(list_in):

    adata_dict = {}
    for num,i in enumerate(list_in):
        adata_dict[i] = num
        
    #adata_dict['Unassigned'] = num + 1
    
    return adata_dict

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P21_V1, adata_t1=P22_glut,test_lab='Type',
                                                                            train_lab='Type', 
                                                                          recomp_HVGs=True, union_hvgs=False,
                                         t0_dict=make_dict(P21_V1, 'Type'),
                                         t1_dict=make_dict(P22_glut, 'Type'),
                                        min_cells=100, x_lab='P22 Sham', y_lab='P21 V1')

test_labelsP12vsP22, test_predlabelsP12vsP22, test_prediction_P12vsP22 = test_items
mappingconfmatP12vsP22, mappingxticksP12vsP22_sub, mappingplotP12vsP22 = mapping_items

In [ ]:
a = make_dict(P22_glut, 'Type')

In [ ]:
a.keys()

In [ ]:
pd.Categorical(test_predlabelsP12vsP22).categories

No  'L5NP_C',  'L5PT_G',, 

In [ ]:
kes = ['L2/3_A', 'L2/3_B', 'L2/3_C', 'L4_A', 'L4_B', 'L4_C', 'L5IT', 'L5NP_A', 'L5NP_B', 'L5PT_A', 'L5PT_B', 
       'L5PT_C', 'L5PT_D', 'L5PT_E', 'L5PT_F', 'L6CT_A', 'L6CT_B', 
       'L6CT_C', 'L6CT_D', 'L6CT_E', 'L6IT_A', 'L6IT_B', 'L6IT_C', 'L6IT_Car3', 'L6b']
P21_V1.obs['P22 Mapping Prob'] = np.max(test_prediction_P12vsP22, axis=1)
P21_V1.obs['P22 Mapping Label'] = pd.Categorical(test_predlabelsP12vsP22).rename_categories(kes)

In [ ]:
freq_scatter(x=P22_glut.obs['Type'].value_counts(normalize=True),
            y= P21_V1.obs['P22 Mapping Label'].value_counts(normalize=True),
            x_lab='P22 Type Freqs', y_lab='P21 V1 Types Mapping Freq', hue_='Types', low_lim=0,unity_lim=0.2, )

In [ ]:
P22_glut.obs['Sub'] = P22_glut.obs['Subclass'].cat.rename_categories(['L2/3', 'L4', 'L4/5IT', 'L5NP', 
                                                                      'L5PT', 'L6CT', 'L6IT', 'L6b'])
P21_V1.obs['Sub'] = P21_V1.obs['Subclass'].cat.rename_categories(['L2/3', 'L4', 'L4/5IT', 'L5NP', 
                                                                      'L5PT', 'L6CT', 'L6IT', 'L6b'])

In [ ]:
c = 0
lims = [0.6, 0.41, 0.25, 0.35]
low_lims = [0.2, 0.1,0,0]
xes = []
yes = []
for i in ['L2/3', 'L4', 'L5', 'L6']:
    print(i)
    a=P22_glut[P22_glut.obs['Sub'].str.startswith(i),:]
    b=P21_V1[P21_V1.obs['Sub'].str.startswith(i),:]
    
    xx = a.obs['Type'].value_counts(normalize=True)
    yy = b.obs['P22 Mapping Label'].value_counts(normalize=True)
    
    #print(b.obs['P22 Mapping Label'].value_counts())
    
    if (i=='L4'):
        freq_scatter(x=xx[0:4], y=yy[0:4],
            x_lab='P22 Type Freqs', y_lab='P21 V1 Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
        oh=xx
        my=yy
        xes.append(xx[0:4])
        yes.append(yy[0:4])
        
    
    else:
        oh=xx
        my=yy
        freq_scatter(x=xx[xx.index.str.startswith(i)],
            y=yy[yy.index.str.startswith(i)],
            x_lab='P22 Type Freqs', y_lab='P21 V1 Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
        xes.append(xx[xx.index.str.startswith(i)])
        yes.append(yy[yy.index.str.startswith(i)])
        
    plt.legend(ncol=1, bbox_to_anchor=(1,1))
    plt.show()
    c = c +1

### GABA

In [ ]:
P21_V1g = rem_fem(sc.read_h5ad('P21full_gaba_v3.h5ad'))
P21_V1g = P21_V1g[P21_V1g.obs['Study']!='2022 RNA',:].copy()
P21_V1g.obs['Type'] = P21_V1g.obs['Type_nn_dists']
P21_V1g.X = P21_V1g.raw.X

In [ ]:
P22_gaba = sc.read_h5ad('h5ads/P22Sham_gaba_v2.h5ad')
P22_gaba.X = P22_gaba.raw.X

**The change below to plot_conf_matx() fixes the indeing issue.**

In [ ]:
#code
#import
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scanpy as sc
import pandas as pd
from harmony import harmonize
from sklearn.metrics.cluster import adjusted_rand_score
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.utils import shuffle
from anndata import AnnData
from sklearn.cluster import KMeans
from typing import Union, Optional, Tuple, Collection, Sequence, Iterable
from scipy.stats import hypergeom
import sklearn.preprocessing
import seaborn as sn
from random import sample
import pickle
#from matplotlib_venn import venn3, venn3_circles
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples


from matplotlib import gridspec
from sklearn.metrics import confusion_matrix, adjusted_rand_score, roc_curve, auc, classification_report, f1_score, cohen_kappa_score
import plotly.graph_objects as go
from itertools import cycle, islice
from sklearn.preprocessing import label_binarize

import scanpy as sc
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.set_figure_params(dpi=100, dpi_save=200)


import scrublet as scr

from typing import Optional

# import warnings
# warnings.filterwarnings("ignore")

import matplotlib as mpl
from matplotlib import gridspec
import xgboost as xgb
from sklearn.metrics import accuracy_score


sc.settings.verbosity = 0           # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.set_figure_params(dpi=75, dpi_save=200)

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

def test_func():
    print('test')
    
def test2():
    print('test2')
def rem_fem(adata):
    for j in list(adata.obs.Type_nn_dists.values.categories):
        if ('Fem' in j):
            adata = adata[adata.obs['Type_nn_dists']!=j,:]
            adata = adata[adata.obs['Type_leiden']!=j,:]
            adata = adata[adata.obs['Type']!=j,:]
    return adata
def pipeline_short(adata, batch_correct, batch_ID):
                   
    if sp.sparse.issparse(adata.X):
        if np.any(adata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(adata.X<0):
            raise Exception("Matrix contains negative values")

    
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch_ID) #HVGs
    adata.raw = adata
    sc.pp.scale(adata, max_value=10) #scale
    sc.tl.pca(adata, svd_solver='arpack') #run PCA
    
    if (batch_correct):
        Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = batch_ID)
        adata.obsm['X_harmony'] = Z
        sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_harmony', n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

    else:
        sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)
        
def pre_process(adata, batch_ID):
    
    if sp.sparse.issparse(adata.X):
        if np.any(adata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(adata.X<0):
            raise Exception("Matrix contains negative values")
    
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch_ID) #HVGs
    adata.raw = adata
    sc.pp.scale(adata, max_value=10) #scale
    sc.tl.pca(adata, svd_solver='arpack') #run PCA
    

def pipeline(adata, 
             batch_correct: Optional[bool] = None,
             batch_ID: Optional[str] = None):
    
    if (batch_correct):
        Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = batch_ID)
        adata.obsm['X_harmony'] = Z
        sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_harmony', n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

    else:
        sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

def clust_obs_plot(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    
    x,y = ser.index, ser.values
    
    plt.figure(figsize=(15,4))
    plt.bar(x,y)
    plt.axhline(np.median(adata.obs[obs_id]), color='r')
    plt.ylabel(obs_id)
    
    return x, y

def DE(adata, obs_id, obs_id_test, ref, pts_thresh, lf_thresh):

    sc.tl.rank_genes_groups(adata, groupby=obs_id, groups=[obs_id_test], 
                                reference=ref, method='t-test', pts=True, use_raw=True)

    lfcs = adata.uns['rank_genes_groups']['logfoldchanges'].astype([(obs_id_test, '<f8')]).view('<f8') 

    l231_genes = adata.uns['rank_genes_groups']['pts']

    lfcs = []
    p_adj = []
    names = list(adata.uns['rank_genes_groups']['names'].astype([(obs_id_test, '<U50')]).view('<U50'))
    logfoldchanges = adata.uns['rank_genes_groups']['logfoldchanges'].astype([(obs_id_test, '<f8')]).view('<f8')
    pvals_adj = adata.uns['rank_genes_groups']['pvals_adj'].astype([(obs_id_test, '<f8')]).view('<f8')

    for i in l231_genes.index:
        lfcs.append(logfoldchanges[names.index(i)])
        p_adj.append(pvals_adj[names.index(i)])

    l231_genes['LF'] = lfcs
    l231_genes['p_adj'] = p_adj
    
    #plt.hist(l231_genes[obs_id_test].values)

    l231_genes = l231_genes[l231_genes[obs_id_test]>pts_thresh]
    
    sort_LF = l231_genes.sort_values('LF', ascending=False)
    
    a = np.where(sort_LF[sort_LF['LF']>0.6]['p_adj'].values<0.05)[0].shape[0]
    b = sort_LF[sort_LF['LF']>0.6].shape[0]
    if(a == b):
        print('cutoffs are good at 1.5 FC level')
    else:
        print(a,b)
        
    return sort_LF[sort_LF['LF']>lf_thresh]
        
def clust_obs(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    return ser.index, ser.values

def res_tune(adata, def_labels):
    
    adata_ari = []
    adata_num_clusts = []
    for res in np.arange(1,2.1, 0.1):
        sc.tl.leiden(adata, resolution=res)

        adata_num_clusts.append(len(adata.obs.leiden.values.categories))
        adata_ari.append(adjusted_rand_score(def_labels, list(adata.obs.leiden.values)))
        
    return adata_num_clusts, adata_ari

from math import log, e
def entropy2(labels, base=None):
  """ Computes entropy of label distribution. """

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value,counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  # Compute entropy
  base = e if base is None else base
  for i in probs:
    ent -= i * log(i, base)

  return ent

def check_clust(adata, clus, ref_age):
    sc.pl.umap(adata[adata.obs.leiden==clus], color=[ref_age+'-2022 Mapping Prob', ref_age+'-2022 Mapping Label'])
    print(adata[adata.obs.leiden==clus].shape, 'mean prob', np.mean(adata[adata.obs.leiden==clus].obs[ref_age+'-2022 Mapping Prob']))
    ser = adata[adata.obs.leiden==clus].obs['Sample'].value_counts()
    plt.bar(ser.index, ser.values,)
    plt.xticks(rotation=90)
    plt.show()
    ser = adata[adata.obs.leiden==clus].obs[ref_age+'-2022 Mapping Label'].value_counts()
    plt.bar(ser.index, ser.values,)
    plt.xticks(rotation=90)
    plt.show()
    
def maj_vote_annot(adata, clust_id, putative_annot, final_annot, vlmcendo=False):
    clust_list = []
    for i in adata.obs[clust_id].values.categories:
        clust = adata[adata.obs[clust_id]==i,:]
        clust_df = clust.obs[putative_annot].value_counts()
        if (clust_df.index[0]=='Unassigned'):
            biggest_cat = clust_df.index[1]
        else: biggest_cat = clust_df.index[0]
        if (vlmcendo):
            if (biggest_cat in ('VLMC', 'Endo')):
                clust.obs[putative_annot+'_maj'] = ['VLMC+Endo']*clust.shape[0]
                clust_list.append(clust)
            else:
                clust.obs[putative_annot+'_maj'] = [biggest_cat]*clust.shape[0]
                clust_list.append(clust)
        else:
            clust.obs[putative_annot+'_maj'] = [biggest_cat]*clust.shape[0]
            clust_list.append(clust)

    adata_annot = clust_list[0].concatenate(clust_list[1:], index_unique=None)
    adata_annot.obs[final_annot] = adata_annot.obs[putative_annot+'_maj']
    a = sc.pl.dotplot(adata_annot, 'Mdga1', 'leiden',) #to fix categories
    return adata_annot

def clus_sample_bars(adata, a_, b_, samp_id, clus_id, size=(13,9.5)):
    adata_ser = adata.obs[samp_id].value_counts(normalize=True)
    plt.bar(adata_ser.index, adata_ser.values)
    plt.xticks(rotation=90)
    plt.title('Overall Dataset')
    plt.show()
    a,b = 0 ,0 
    fig, axs = plt.subplots(a_,b_, figsize=size)

    for i in adata.obs[clus_id].values.categories:
        clus = adata[adata.obs[clus_id]==i,:]
        clus_ser = clus.obs[samp_id].value_counts(normalize=True)
        axs[a,b].bar(clus_ser.index, clus_ser.values)
        axs[a,b].set_title(i+' ('+str(clus.shape[0])+' cells)')
        axs[a,b].set_ylabel('Fraction')
        axs[a,b].set_xticklabels(clus_ser.index, rotation=90)
        b = b + 1
        if (b>=b_):
            b = 0
            a = a + 1
    plt.tight_layout()
    
    
def euclidean_distance(vector1, vectors_list):
    # Ensure that the input vectors are numpy arrays
    vector1 = np.array(vector1)
    vectors_list = [np.array(vector) for vector in vectors_list]
    
    # Calculate the Euclidean distance between the input vector and each vector in the list
    distances = [np.linalg.norm(vector1 - vector) for vector in vectors_list]
    return distances

def nn_voting(adata, type_old, type_new, delta_thresh):
    adata.obs['idx'] = np.arange(adata.shape[0])

#     #re-make graph using UMAP space for voting below
#     sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_umap')
#     sc.tl.umap(adata)

    adata.obs[type_new] = adata.obs[type_old]
    adata_un_idx = adata[adata.obs[type_old]=='Unassigned',:].obs.idx.values #indices of unassigned cells
    neigh_matx = adata.obsp['distances'].A #each row's nonzero entries tells neighbors
    adata_iGBs = list(adata.obs[type_old].values) #all the step1 assignments
    n_neighbs = adata.uns['neighbors']['params']['n_neighbors']
    print(adata.uns['neighbors']['params'])
    print(' ')
    
    print("Pre-voting unassigned: ", adata_iGBs.count('Unassigned'),
          adata_iGBs.count('Unassigned')/adata.shape[0])
    
    
    delta = 1 #init delta
    while (delta>delta_thresh):    
        un_frac1 = adata_iGBs.count('Unassigned')/adata.shape[0] #frac of unassigned cells before voting

        #loop thru each cell
        for i in adata_un_idx:

            #so that it only loops thru still-unassigned cells after first pass
            if (adata.obs[type_new][i]=='Unassigned'):
                neighbs_idx = np.where(neigh_matx[i,:]>0)[0] #cell i's neighbors
                neighbs_iGBs = adata[neighbs_idx].obs[type_new] #the neighbors' iGBs

#                 #if there's a type in the neighbors that is majority, assign it
#                 if (neighbs_iGBs.value_counts()[0] > n_neighbs/2):
#                     adata_iGBs[i] = neighbs_iGBs.value_counts().index[0]
                
                #no need to be majority, just be biggest one
                adata_iGBs[i] = neighbs_iGBs.value_counts().index[0]
                #update the IDs to help assignment of next cell
                adata.obs[type_new] =  pd.Categorical(adata_iGBs)

        un_frac2 = adata_iGBs.count('Unassigned')/adata.shape[0] #frac of unassigned cells after voting

        delta = (un_frac1-un_frac2)/un_frac1   #stop when this changes by less than 1%
        print(delta, un_frac2)

    print("Post-voting unassigned: ", adata_iGBs.count('Unassigned'),
          adata_iGBs.count('Unassigned')/adata.shape[0])
    
    return adata


def make_hmap(adata,thres, y_obs='Type', x_obs='leiden', ):
    crosstab_data = pd.crosstab(adata.obs[y_obs], adata.obs[x_obs], 
                           normalize='index')
    # For rows
    row_order = np.argsort(-crosstab_data.values.max(axis=1))

    # For columns
    column_order = np.argsort(-crosstab_data.values.max(axis=0))

    reordered_rows = crosstab_data.index[row_order]
    reordered_crosstab_rows = crosstab_data.iloc[row_order]

    # For columns
    reordered_columns = crosstab_data.columns[column_order]
    reordered_crosstab = reordered_crosstab_rows[reordered_columns]
    #return reordered_crosstab
    
    
    from sklearn.metrics import adjusted_rand_score

    

    labels_true = adata.obs[y_obs]
    labels_pred = adata.obs[x_obs]

    # Compute the Adjusted Rand Index
    ari = adjusted_rand_score(labels_true, labels_pred)
    
    return [reordered_crosstab.applymap(lambda value: value if value > thres else np.round(value, 2)),
           ari]
def freq_scatter(x, y, x_lab, y_lab, hue_, unity_lim, low_lim):

    if (x.shape[0]!=y.shape[0]): 
        print('x and y are different by', np.abs(x.shape[0]-y.shape[0]), 'categories')
    
    if (y.shape[0]>x.shape[0]):
        y = y[x.index]
    else:
        x = x[y.index]

    df = pd.DataFrame(index=list(x.index.values), columns=[x_lab, y_lab],
                      data=np.transpose(np.array([x.values, y.values])))

    df[hue_] = df.index
    sn.scatterplot(data=df, x=x_lab, y = y_lab, hue=hue_, s=50)
    plt.legend(bbox_to_anchor=(1.01, 1.03), loc='upper left', fontsize=14, ncol=2)
    plt.plot(np.linspace(low_lim,unity_lim), np.linspace(low_lim,unity_lim), ls='--', 
             color='black', linewidth=0.75)
    plt.title('Pearson R: '+str(np.round(sp.stats.pearsonr(x, y)[0], 3)))
    plt.grid(False)
    #plt.loglog()
    


from anndata import AnnData
from typing import Union, Optional, Tuple, Collection, Sequence, Iterable

def module_score(adata:AnnData, genes_use: list, score_name: Optional[str] = None, verbose: bool = True):
    
    """\
    Compute module scores for all cells in adata as described in methods of RGC-dev paper.
    
    
    Parameters
    ----------
    adata
        The (annotated) data matrix of shape `n_obs` × `n_vars`.
        Rows correspond to cells and columns to genes.
    genes_use
        list of genes in module of interest
    score_name
        Name endowed to the module score to be computed
        e.g. "Mod1"
    verbose
        Inform user of fraction of module genes that are in adata
        
    Returns
    -------
    adata with a new .obs called score_name
    
    """
    
    if (score_name==None):
        score_name = str(input("Provide a name for this score (no spaces): "))
        
    genes_use0 = genes_use
    genes_use = list(set(genes_use).intersection(adata.var_names))#genes that are both in module and `adata`
    
     
    if (len(genes_use) == 0):
        raise ValueError("Error : Must provide a list of genes that are present in the data")
        
    
    if (verbose):
        if(len(genes_use0) > len(genes_use)):
            n = len(genes_use0) - len(genes_use)
            print(score_name,": Note that", n, "of the", len(genes_use0), "genes in your module do not exist in the data set." )
        else:
            print(score_name,": Note that all of the", len(genes_use), "genes in your module are in the data set." )
    
    
    
    adata_score = adata.copy()
    adata_score = adata[:,genes_use]
    
    counts_modgenes = adata_score.X.toarray() #all cells, module genes
    counts_all = adata.X.toarray() #all cells, all genes
    #scores = np.mean(counts_modgenes, axis=1) - np.mean(counts_all, axis=1) #(row means of counts_modgenes ) - (row means of counts_all)
    scores = np.mean(counts_modgenes, axis=1) #(row means of counts_modgenes )

    adata.obs[score_name] = scores
    
    return genes_use    
    

def clean_subclasses(adata, rep):
    #define avg PC position of each type. Gotta use a type with no "Unassigned" group
    typ_mean_dict = {}
    for i in adata.obs['Subclass'].values.categories:
        typ_mean_dict[i] = np.mean(adata[adata.obs['Subclass']==i,:].obsm[rep][:,0:40], axis=0)

    #assign based on proximity to avg type
    dists_list = []
    dict_keys = list(typ_mean_dict.keys())
    for i in range(adata.shape[0]):
        typ_ = adata.obs['Subclass'][i]
        dists = euclidean_distance(adata.obsm[rep][i,0:40], list(typ_mean_dict.values()))

        #dist = dists[dict_keys.index(typ_)]
        typ = dict_keys[np.argmin(dists)]
        #types.append(typ)
        if (typ!=typ_):
            dists_list.append(100)
        else: dists_list.append(0)

    adata.obs['Dist to Subclass'] = dists_list
    
def make_dict(adata, obs_id):

    adata_dict = {}
    for num,i in enumerate(adata.obs[obs_id].values.categories):
        adata_dict[i] = num
        
    #adata_dict['Unassigned'] = num + 1
    
    return adata_dict
    

def plotConfusionMatrix(
    ytrue,
    ypred,
    type,
    xaxislabel,
    yaxislabel,
    title,
    train_dict,
    test_dict=None,
    re_order=None,
    re_order_cols = None,
    re_index = None,
    re_order_rows = None,
    save_as=None,):
    
    #very bad
    numbertrainclasses = len(set(ypred))
    numbertestclasses = len(set(ytrue))
    
    #cfm is 11x11 b/c 11 is = y_true U y_pred
    confusionmatrix = confusion_matrix(y_true = ytrue, y_pred = ypred)
        
    #only need this when mapping b/c if validaiton, all classes will be used and cfm will be constructed properly
    if type == 'mapping':
        rows = np.where(np.sum(confusionmatrix, axis=1)>0)[0]
        cols = np.where(np.sum(confusionmatrix, axis=0)>0)[0]
        
        cfm = confusionmatrix[rows,:][:,cols]
        
        #show all columns, even ones with no mapping
        
        #this was changed 10/12/2023 to address unassigned issue
        cfm_z = np.zeros((len(test_dict),len(train_dict)))
        #cfm_z = np.zeros((len(rows),len(cols)))

        cfm_z[:, np.array(pd.Categorical(ypred).categories, dtype='int')]=cfm 
        confusionmatrix = cfm_z
        
        #always keep only as many as rows as num of test classes
        #but, b/c of python's 0 indexing, if the number of training classes is in the 
        #y_pred list, then that means there was an unassigned
#       if numbertrainclasses in ypred:
#         confusionmatrix = confusionmatrix[0:numbertestclasses,0:numbertrainclasses+1]#for Unassigned
#       else:
#         confusionmatrix = confusionmatrix[0:numbertestclasses,0:numbertrainclasses]
    
        confmatpercent = confusionmatrix/np.sum(confusionmatrix, axis=1).reshape(-1,1)

        conf_df = pd.DataFrame(confmatpercent)
        conf_df.index = list(test_dict.keys())

        #name columns of conf mat
        if(len(conf_df.columns)>len(train_dict)):
            conf_df.columns = list(train_dict.keys())+['Unassigned']
        else:
            conf_df.columns = list(train_dict.keys())


        if (re_order):
            conf_df = conf_df[re_order_cols]
        if (re_index):
            conf_df = conf_df.reindex(re_order_rows)

        diagcm = conf_df.to_numpy()
    
    
        xticksactual = list(conf_df.columns)
        
        #print(conf_df)
        
    
    else:
        confmatpercent = np.zeros(confusionmatrix.shape)
        for i in range(confusionmatrix.shape[0]):
            if np.sum(confusionmatrix[i,:]) != 0:
                confmatpercent[i,:] = confusionmatrix[i,:]/np.sum(confusionmatrix[i,:])
            else:
                confmatpercent[i,:] = confusionmatrix[i,:]
            diagcm = confmatpercent
            xticks = np.linspace(0, confmatpercent.shape[1]-1, confmatpercent.shape[1], dtype = int)
        xticksactual = []
        for i in xticks:
            if i != numbertrainclasses:
                xticksactual.append(list(train_dict.keys())[i])
            else:
                xticksactual.append('Unassigned')
        
    dot_max = np.max(diagcm.flatten())
    dot_min = 0
    if dot_min != 0 or dot_max != 1:
        frac = np.clip(diagcm, dot_min, dot_max)
        old_range = dot_max - dot_min
        frac = (frac - dot_min) / old_range
    else:
        frac = diagcm
    xvalues = []
    yvalues = []
    sizes = []
    for i in range(diagcm.shape[0]):
        for j in range(diagcm.shape[1]):
            xvalues.append(j)
            yvalues.append(i)
            sizes.append((frac[i,j]*35)**1.5)
    size_legend_width = 0.5
    height = diagcm.shape[0] * 0.3 + 1
    height = max([1.5, height])
    heatmap_width = diagcm.shape[1] * 0.35
    width = (
        heatmap_width
        + size_legend_width
        )
    fig = plt.figure(figsize=(width, height))
    axs = gridspec.GridSpec(
        nrows=2,
        ncols=2,
        wspace=0.02,
        hspace=0.04,
        width_ratios=[
                    heatmap_width,
                    size_legend_width
                    ],
        height_ratios = [0.5, 10]
        )
    dot_ax = fig.add_subplot(axs[1, 0])
    dot_ax.scatter(xvalues,yvalues, s = sizes, c = 'blue', norm=None, edgecolor='none')
    y_ticks = range(diagcm.shape[0])
    dot_ax.set_yticks(y_ticks)
    if type == 'validation':
        dot_ax.set_yticklabels(list(train_dict.keys()))
    elif type == 'mapping':
      #dot_ax.set_yticklabels(list(test_dict.keys()))
        dot_ax.set_yticklabels(list(conf_df.index))
    x_ticks = range(diagcm.shape[1])
    dot_ax.set_xticks(x_ticks)
    dot_ax.set_xticklabels(xticksactual, rotation=90)
    dot_ax.tick_params(axis='both', labelsize='small')
    dot_ax.grid(True, linewidth = 0.2)
    dot_ax.set_axisbelow(True)
    dot_ax.set_xlim(-0.5, diagcm.shape[1] + 0.5)
    ymin, ymax = dot_ax.get_ylim()
    dot_ax.set_ylim(ymax + 0.5, ymin - 0.5)
    dot_ax.set_xlim(-1, diagcm.shape[1])
    dot_ax.set_xlabel(xaxislabel)
    dot_ax.set_ylabel(yaxislabel)
    dot_ax.set_title(title)
    size_legend_height = min(1.75, height)
    wspace = 10.5 / width
    axs3 = gridspec.GridSpecFromSubplotSpec(
        2,
        1,
        subplot_spec=axs[1, 1],
        wspace=wspace,
        height_ratios=[
                    size_legend_height / height,
                    (height - size_legend_height) / height
                    ]
        )
    diff = dot_max - dot_min
    if 0.3 < diff <= 0.6:
        step = 0.1
    elif diff <= 0.3:
        step = 0.05
    else:
        step = 0.2
    fracs_legends = np.arange(dot_max, dot_min, step * -1)[::-1]
    if dot_min != 0 or dot_max != 1:
        fracs_values = (fracs_legends - dot_min) / old_range
    else:
        fracs_values = fracs_legends
    size = (fracs_values * 35) ** 1.5
    size_legend = fig.add_subplot(axs3[0])
    size_legend.scatter(np.repeat(0, len(size)), range(len(size)), s=size, c = 'blue')
    size_legend.set_yticks(range(len(size)))
    labels = ["{:.0%}".format(x) for x in fracs_legends]
    if dot_max < 1:
        labels[-1] = ">" + labels[-1]
    size_legend.set_yticklabels(labels)
    size_legend.set_yticklabels(["{:.0%}".format(x) for x in fracs_legends])
    size_legend.tick_params(axis='y', left=False, labelleft=False, labelright=True)
    size_legend.tick_params(axis='x', bottom=False, labelbottom=False)
    size_legend.spines['right'].set_visible(False)
    size_legend.spines['top'].set_visible(False)
    size_legend.spines['left'].set_visible(False)
    size_legend.spines['bottom'].set_visible(False)
    size_legend.grid(False)
    ymin, ymax = size_legend.get_ylim()
    size_legend.set_ylim(ymin, ymax + 0.5)
    
    if (save_as is not None):
        fig.savefig(save_as, bbox_inches = 'tight')

    return diagcm, xticksactual, axs



#This helper method plots validation plots in sequential order (i.e. first plot is for first batch, second plot is for second batch, etc.)
def plot_validation_plots(validation_label_train_70, valid_predlabels_train_70, train_dict, save_as=None):
    
    ARI = adjusted_rand_score(labels_true = validation_label_train_70, 
                              labels_pred = valid_predlabels_train_70)
    
    
    c = 0
    for i in range(validation_label_train_70.shape[0]):
        if (validation_label_train_70[i]!=valid_predlabels_train_70[i]):
            c = c +1
    acc = (1 - c/len(validation_label_train_70))*100
    
           
    validationconfmat, validationxticks, validationplot = plotConfusionMatrix(
    ytrue = validation_label_train_70,
    ypred = valid_predlabels_train_70,
    train_dict=train_dict,
    type = 'validation',
    save_as = save_as,
    title = 'ARI = {:.3f}, Accuracy = {:.3f}'.format(ARI, acc),
    xaxislabel = 'Predicted',
    yaxislabel = 'True'
    )

def plot_mapping(test_labels, test_predlabels, test_dict, train_dict, 
                 xaxislabel, yaxislabel,
                re_order=None,
    re_order_cols = None,
                 re_index = None,
    re_order_rows = None, save_as=None):
    
    ARI = adjusted_rand_score(labels_true = test_labels, 
                              labels_pred = test_predlabels)
    NCE = calculateNCE(labels_true = test_labels, labels_pred = test_predlabels)
    
 
    
           
    mappingconfmat, mappingxticks, mappingplot = plotConfusionMatrix(
    ytrue = test_labels,
    ypred = test_predlabels,
    test_dict=test_dict,
    train_dict=train_dict,
    type = 'mapping',
    save_as = save_as,
    title = 'ARI = {:.3f}, NCE = {:.3f}'.format(ARI, NCE),
    xaxislabel =xaxislabel,
    yaxislabel = yaxislabel,
        re_order=re_order,
    re_order_cols = re_order_cols,
        re_index = re_index,
    re_order_rows = re_order_rows,
    ) 
    return mappingconfmat, mappingxticks, mappingplot
      
      
#This helper method uses xgboost to train classifiers.
def trainclassifier(train_anndata, common_top_genes, obs_id, train_dict, eta,
                    max_cells_per_ident, train_frac, min_cells_per_ident):
    
    if sp.sparse.issparse(train_anndata.X):
        if np.any(train_anndata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(train_anndata.X<0):
            raise Exception("Matrix contains negative values")

    
    start_time = time.time()
    
    numbertrainclasses = len(train_anndata.obs[obs_id].values.categories)

    xgb_params_train = {
            'objective':'multi:softprob',
            'eval_metric':'mlogloss',
            'num_class':numbertrainclasses,
            'eta':eta,
            'max_depth':4,
            'subsample': 0.6}
    nround = 200
    #Train XGBoost on 70% of training data and validate on the remaining data


    training_set_train_70 = []
    validation_set_train_70 = []
    training_label_train_70 = []
    validation_label_train_70 = []

    #loop thru classes to split for training and validation
    for i in train_anndata.obs[obs_id].values.categories:
        
        #how many cells in a class
        cells_in_clust = train_anndata[train_anndata.obs[obs_id]==i,:].obs_names #cell names
        n = min(max_cells_per_ident,round(len(cells_in_clust)*train_frac))
        
        #sample 70% for training and rest for validation
        train_temp = np.random.choice(cells_in_clust,n,replace = False)
        validation_temp = np.setdiff1d(cells_in_clust, train_temp)
        
        #upsample small clusters
        if len(train_temp) < min_cells_per_ident:
            train_temp_bootstrap = np.random.choice(train_temp, size = min_cells_per_ident - int(len(train_temp)))
            train_temp = np.hstack([train_temp_bootstrap, train_temp])
        
        #store training and validation **names** of cells in vectors, which update for every class
        training_set_train_70 = np.hstack([training_set_train_70,train_temp])
        validation_set_train_70 = np.hstack([validation_set_train_70,validation_temp])
        
        #store training and validation **labels** of cells in vectors, which update for every class
        training_label_train_70 = np.hstack([training_label_train_70,np.repeat(train_dict[i],len(train_temp))])
        validation_label_train_70 = np.hstack([validation_label_train_70,np.repeat(train_dict[i],len(validation_temp))])

        #need train_dict b/c XGboost needs number as class labels, not words
        #this is only deconvulted later in plotting function
        
    #put data in XGB format
    X_train = train_anndata[training_set_train_70,common_top_genes].X
    train_matrix_train_70 = xgb.DMatrix(data = X_train, label = training_label_train_70, 
                                        feature_names = common_top_genes)
    
    X_valid = train_anndata[validation_set_train_70,common_top_genes].X
    validation_matrix_train_70 = xgb.DMatrix(data = X_valid, label = validation_label_train_70, 
                                             feature_names = common_top_genes)

    del training_set_train_70, validation_set_train_70, training_label_train_70
    
    #Train on 70%
    bst_model_train_70 = xgb.train(
        params = xgb_params_train,
        dtrain = train_matrix_train_70,
        num_boost_round = nround)
    
    #Validate on 30%
    #a validation_cells x numclasses matrix, with each vector containing prob association with the classes
    validation_pred_train_70 = bst_model_train_70.predict(data = validation_matrix_train_70)
    
    #for each cell, go through vec of probs and take index of max prob: that's assignment
    valid_predlabels_train_70 = np.zeros((validation_pred_train_70.shape[0]))
    for i in range(validation_pred_train_70.shape[0]):
        valid_predlabels_train_70[i] = np.argmax(validation_pred_train_70[i,:])
        
    
    #Train on 100%
    #Train XGBoost on the full training data
    training_set_train_full = []
    training_label_train_full = []

    for i in train_anndata.obs[obs_id].values.categories.values:
        train_temp = train_anndata.obs.index[train_anndata.obs[obs_id].values == i]
        if len(train_temp) < 100:
            train_temp_bootstrap = np.random.choice(train_temp, size = 100 - int(len(train_temp)))
            train_temp = np.hstack([train_temp_bootstrap, train_temp])
        
        #indices of cells in class
        training_set_train_full = np.hstack([training_set_train_full,train_temp])
        
        #labels of cells in class: [label*N_class] stacked onto previous classes
        training_label_train_full = np.hstack([training_label_train_full,np.repeat(train_dict[i],len(train_temp))])


    X_train_full = train_anndata[training_set_train_full,common_top_genes].X
    full_training_data = xgb.DMatrix(data = X_train_full, label = training_label_train_full, 
                                     feature_names = common_top_genes)

    del training_set_train_full, training_label_train_full

    bst_model_full_train = xgb.train(
        params = xgb_params_train,
        dtrain = full_training_data,
        num_boost_round = nround)

    
    
    print('trainclassifier() complete after', np.round(time.time() - start_time), 'seconds')
    
    
    f1 = f1_score(validation_label_train_70, valid_predlabels_train_70, average = None)

    
    #real labels of validation set, predicted labels, classifier.
    #recall these are all integers that are deconvulted later in plotting using the dicts
    return validation_label_train_70, valid_predlabels_train_70, bst_model_full_train, f1


#This helper method predicts the testing cluster labels.
def predict(train_anndata, common_top_genes, bst_model_train_full, test_anndata, 
            train_obs_id, test_dict, test_obs_id):
    
    
    if sp.sparse.issparse(train_anndata.X):
        if np.any(train_anndata.X.A<0):
            raise Exception("Training matrix contains negative values")
    else: 
        if np.any(train_anndata.X<0):
            raise Exception("Training matrix contains negative values")

    if sp.sparse.issparse(test_anndata.X):
        if np.any(test_anndata.X.A<0):
            raise Exception("Testing matrix contains negative values")
    else: 
        if np.any(test_anndata.X<0):
            raise Exception("Testing matrix contains negative values")
  
    
    #Predict the testing cluster labels
    #how many classes mapping to 
    numbertrainclasses = len(train_anndata.obs[train_obs_id].values.categories)
    
    #put testing data into XGB format
    full_testing_data = xgb.DMatrix(data = test_anndata[:,common_top_genes].X, 
                                    feature_names=common_top_genes)
    
    #a testing_cells x numclasses matrix, with each vector containing prob association with the classes
    test_prediction = bst_model_train_full.predict(data = full_testing_data)

    #for each cell, go through vec of probs and take index of max prob (if greater than ...): that's assignment

    
    test_predlabels = np.zeros((test_prediction.shape[0]))
    for i in range(test_prediction.shape[0]):
        if np.max(test_prediction[i, :]) > 1.1*(1/numbertrainclasses):
            test_predlabels[i] = np.argmax(test_prediction[i,:])
        
        #"unassigned" is a label one larger than all b/c python begins indexing at 0
        else:
            test_predlabels[i] = numbertrainclasses
        
    test_labels = np.zeros(len(test_anndata.obs[test_obs_id].values))
    for i,l in enumerate(test_anndata.obs[test_obs_id].values):
        test_labels[i] = test_dict[l]

    #actual labels of testing set, the labels that test set mapped to 
    return test_labels, test_predlabels, test_prediction

def calculateNCE(labels_true,labels_pred):
    X = labels_true
    Y = labels_pred
    contTable = confusion_matrix(X,Y)[0:len(np.unique(X)), 0:len(np.unique(Y))]
    a = np.sum(contTable, axis = 1)
    b = np.sum(contTable, axis = 0)
    N = np.sum(contTable)
    pij = contTable/N
    pi = a/N
    pj = b/N
    Hyx = np.zeros(contTable.shape)
    for i in range(contTable.shape[0]):
        for j in range(contTable.shape[1]):
          if pij[i,j] == 0:
            Hyx[i,j] = 0
          else:
            Hyx[i,j] = pij[i,j]*np.log10(pij[i,j]/pi[i])
    CE = -np.sum(Hyx)
    Hyi = np.zeros(contTable.shape[1])
    for j in range(contTable.shape[1]):
      if pj[j] == 0:
       Hyi[j] = 0
      else:
        Hyi[j] = pj[j]*np.log10(pj[j])
    Hy = -np.sum(Hyi)
    NCE = CE/Hy
    return NCE

def train_validate(adata, adata_cell,preproc=False):
    if (preproc):
        adata_cell.X = adata_cell.raw.X
        sc.pp.highly_variable_genes(adata_cell, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs

    common_hvgs = list(set(adata[:,adata.var.highly_variable].var_names).intersection(set(adata_cell.var_names)))
    adata_m_dict = make_dict(adata, obs_id='leiden')
    adata_cell_dict = make_dict(adata_cell, obs_id='Subclass')
    print(len(common_hvgs), 'Shared HVGs')

    valid_truelabel_adata_cell, valid_predlabels_adata_cell, model_atlas_adata_cell = trainclassifier(train_anndata=adata_cell, 
                                                                                                 common_top_genes=common_hvgs, 
                                                                                                 obs_id='Subclass', 
                                                                                                 train_dict=adata_cell_dict, 
                                                                                                 eta=0.2,
                                                                                                 max_cells_per_ident=1000, 
                                                                                          train_frac=0.7, 
                                                                                                 min_cells_per_ident=100)

    plot_validation_plots(valid_truelabel_adata_cell, valid_predlabels_adata_cell, train_dict=adata_cell_dict)
    
    return model_atlas_adata_cell

def pairwise_map(adata_t0, adata_t1, test_lab, train_lab, t0_dict, t1_dict, recomp_HVGs, min_cells,
                x_lab, y_lab, union_hvgs):
    adata_t0 = adata_t0.copy() 
    adata_t1 = adata_t1.copy() 
    adata_t0.X = adata_t0.raw.X
    adata_t1.X = adata_t1.raw.X
    
    if (recomp_HVGs):
        sc.pp.highly_variable_genes(adata_t0, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs
        sc.pp.highly_variable_genes(adata_t1, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs
        t0_hvgs = list(adata_t0[:, adata_t0.var.highly_variable].var_names)
        t1_hvgs = list(adata_t1[:, adata_t1.var.highly_variable].var_names)
        
        if (union_hvgs):
            t0_t1hvgs_ = list(set(t0_hvgs).union(t1_hvgs))
            data_inter = list(set(adata_t1.var_names).intersection(set(adata_t0.var_names)))
            t0_t1hvgs = list(set(data_inter).intersection(set(t0_t1hvgs_)))

        else:
            t0_t1hvgs = list(set(t0_hvgs).intersection(t1_hvgs))

    
    else:
        t0_hvgs = list(adata_t0[:, adata_t0.var.highly_variable].var_names)
        t1_hvgs = list(adata_t1[:, adata_t1.var.highly_variable].var_names)
        t0_t1hvgs = list(set(t0_hvgs).intersection(t1_hvgs))

    print(len(t0_t1hvgs), 'shared HVGs')
    
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1,model_f1 = trainclassifier(train_anndata=adata_t1, 
                                                                                                 common_top_genes=t0_t1hvgs, 
                                                                                                 obs_id=train_lab, 
                                                                                                 train_dict=t1_dict, 
                                                                                                 eta=0.2,
                                                                                                 max_cells_per_ident=1000, 
                                                                                          train_frac=0.7, 
                                                                                                min_cells_per_ident=min_cells)

    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=t1_dict)

    test_labelst0vst1, test_predlabelst0vst1, test_prediction_t0vst1 = predict(train_anndata=adata_t1, 
                                                                     common_top_genes=model_t1.feature_names, 
                                                                     bst_model_train_full=model_t1, 
                                                                     test_anndata=adata_t0,
                                                                     train_obs_id=train_lab, 
                                                                     test_dict=t0_dict, 
                                                                     test_obs_id=test_lab)

    mappingconfmatt0vst1, mappingxtickst0vst1, mappingplott0vst1 = plot_mapping(test_labels=test_labelst0vst1, 
                 test_predlabels=test_predlabelst0vst1, 
                 test_dict=t0_dict, 
                 train_dict=t1_dict,
                     xaxislabel=x_lab, yaxislabel=y_lab,)
    
    test_items = test_labelst0vst1, test_predlabelst0vst1, test_prediction_t0vst1
    mapping_items = mappingconfmatt0vst1, mappingxtickst0vst1, mappingplott0vst1
    
    return test_items, mapping_items

def make_colors(PX_dict):
    PX_colors = []
    for i in PX_dict:
        for j in colors_subclass:
            if (j in i):
                PX_colors.append(colors_subclass[j])
    return PX_colors

def rem_fem(adata):
    for j in list(adata.obs.Type_nn_dists.values.categories):
        if ('Fem' in j):
            adata = adata[adata.obs['Type_nn_dists']!=j,:]
            adata = adata[adata.obs['Type_leiden']!=j,:]
    return adata

def list2dict(list_in):

    adata_dict = {}
    for num,i in enumerate(list_in):
        adata_dict[i] = num
        
    #adata_dict['Unassigned'] = num + 1
    
    return adata_dict

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P21_V1g, adata_t1=P22_gaba,test_lab='Type',
                                                                            train_lab='Type', 
                                                                          recomp_HVGs=True, union_hvgs=False,
                                         t0_dict=make_dict(P21_V1g, 'Type'),
                                         t1_dict=make_dict(P22_gaba, 'Type'),
                                        min_cells=100, x_lab='P22 Sham', y_lab='P21 V1')

test_labelsP12vsP22, test_predlabelsP12vsP22, test_prediction_P12vsP22 = test_items
mappingconfmatP12vsP22, mappingxticksP12vsP22_sub, mappingplotP12vsP22 = mapping_items

In [ ]:
a = make_dict(P22_gaba, 'Type')

In [ ]:
a

In [ ]:
a.keys()

In [ ]:
pd.Categorical(test_predlabelsP12vsP22).categories

No   'Lamp5_D', , 'Sst_D',

In [ ]:
kes = ['Lamp5_A', 'Lamp5_B', 'Lamp5_C','Pvalb_A', 'Pvalb_B', 'Pvalb_C', 'Pvalb_D', 
       'Pvalb_E', 'Pvalb_F', 'Sst_A', 'Sst_B', 'Sst_C',  'Sst_E', 'Sst_F', 'Sst_G', 
       'Sst_H', 'Sst_I', 'Sst_J', 'Vip_A', 'Vip_B', 'Vip_C', 'Vip_D', 'Vip_E']
P21_V1g.obs['P22 Mapping Prob'] = np.max(test_prediction_P12vsP22, axis=1)
P21_V1g.obs['P22 Mapping Label'] = pd.Categorical(test_predlabelsP12vsP22).rename_categories(kes)

In [ ]:
freq_scatter(x=P22_gaba.obs['Type'].value_counts(normalize=True),
            y= P21_V1g.obs['P22 Mapping Label'].value_counts(normalize=True),
            x_lab='P22 Type Freqs', y_lab='P21 V1 Types Mapping Freq', hue_='Types', low_lim=0,unity_lim=0.15, )

In [ ]:
c = 0
lims = [0.4, 0.2, 0.6, 0.6]
low_lims = [0, 0,0,0]
xes = []
yes = []
for i in ['Pvalb', 'Sst', 'Vip', 'Lamp5']:
    print(i)
    a=P22_gaba[P22_gaba.obs['Subclass'].str.startswith(i),:]
    b=P21_V1g[P21_V1g.obs['Subclass'].str.startswith(i),:]
    
    xx = a.obs['Type'].value_counts(normalize=True)
    yy = b.obs['P22 Mapping Label'].value_counts(normalize=True)
    
    #print(b.obs['P22 Mapping Label'].value_counts())
    
    if (i=='L4'):
        freq_scatter(x=xx[0:4], y=yy[0:4],
            x_lab='P22 Type Freqs', y_lab='P21 V1 Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
        oh=xx
        my=yy
        xes.append(xx[0:4])
        yes.append(yy[0:4])
        
    
    else:
        oh=xx
        my=yy
        freq_scatter(x=xx[xx.index.str.startswith(i)],
            y=yy[yy.index.str.startswith(i)],
            x_lab='P22 Type Freqs', y_lab='P21 V1 Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
        xes.append(xx[xx.index.str.startswith(i)])
        yes.append(yy[yy.index.str.startswith(i)])
        
    plt.legend(ncol=1, bbox_to_anchor=(1,1))
    plt.show()
    c = c +1

## Final Filtering

For glut, remove L5PT_F, L5PT_G, L6CT_D, and L6CT_E

In [ ]:
P22_glut.shape

In [ ]:
for i in ['L5PT_F', 'L5PT_G', 'L6CT_D', 'L6CT_E']:
    P22_glut = P22_glut[P22_glut.obs['Type']!=i,:].copy()

In [ ]:
P22_glut.shape

For gaba, remove the following

In [ ]:
P22_gaba.shape

In [ ]:
for i in ['Lamp5_D', 'Pvalb_E', 'Sst_D', 'Vip_E']:
    P22_gaba = P22_gaba[P22_gaba.obs['Type']!=i,:].copy()

In [ ]:
P22_gaba.shape

In [ ]:
(28334 - 28090)+(4888-4332)

In [ ]:
P22_glut.write_h5ad('h5ads/P22Sham_glut_v3.h5ad')
P22_gaba.write_h5ad('h5ads/P22Sham_gaba_v3.h5ad')

P22_glut.write_h5ad('h5ads_cmprsd/P22Sham_glut_v3.h5ad')
P22_gaba.write_h5ad('h5ads_cmprsd/P22Sham_gaba_v3.h5ad')

In [ ]:
P22_glut = sc.read_h5ad('h5ads_cmprsd/P22Sham_glut_v3.h5ad')
P22_gaba = sc.read_h5ad('h5ads_cmprsd/P22Sham_gaba_v3.h5ad')

In [ ]:
len(P22_glut.obs['Type'].cat.categories)+len(P22_gaba.obs['Type'].cat.categories)

## UMAPs

In [ ]:
P22_glut = sc.read_h5ad('h5ads_cmprsd/P22Sham_glut_v3.h5ad')
P22_gaba = sc.read_h5ad('h5ads_cmprsd/P22Sham_gaba_v3.h5ad')

In [ ]:
sc.pl.umap(shuffle(P22_glut), color=['Sample'],  )
sc.pl.umap(P22_glut, color=['Type', 'Subclass', 'leiden'], legend_loc='on data',  legend_fontsize=10,
          add_outline=True,outline_width=(0.2,0.05), s=5 , legend_fontoutline=1, legend_fontweight='semibold')

In [ ]:
P22_glut.shape

In [ ]:
pipeline_short(P22_glut, batch_correct=True, batch_ID='Sample')

In [ ]:
sc.pl.umap(P22_glut, color=['Type',], legend_loc='on data',  legend_fontsize=10,
          add_outline=True,outline_width=(0.2,0.05), legend_fontoutline=1, legend_fontweight='semibold',
           palette=list(sn.color_palette("pastel6").as_hex())+list(sn.color_palette("pastel").as_hex()),
          title='P22 Control: 28,090 Glutamatergic Neurons', frameon=True,
          save='P22sham_gluta_FigS3.pdf')

In [ ]:
pipeline_short(P22_gaba, batch_correct=True, batch_ID='Sample')

In [ ]:
P22_gaba.obs['Type'] = P22_gaba.obs['Type'].cat.rename_categories(['Lamp5_A', 'Lamp5_B', 'Lamp5_C', 'Pvalb_A', 'Pvalb_B', 'Pvalb_C',
       'Pvalb_D', 'Pvalb_E', 'Sst_A', 'Sst_B', 'Sst_C', 'Sst_D', 'Sst_E',
       'Sst_F', 'Sst_G', 'Sst_H', 'Sst_I', 'Vip_A', 'Vip_B', 'Vip_C', 'Vip_D',
       'Vip_E'])

In [ ]:
P22_gaba.write_h5ad('h5ads/P22Sham_gaba_v3.h5ad')
P22_gaba.write_h5ad('h5ads_cmprsd/P22Sham_gaba_v3.h5ad')

In [ ]:
sc.pl.umap(P22_gaba, color=['Type',], legend_loc='on data',  legend_fontsize=10,
          add_outline=True,outline_width=(0.2,0.05), legend_fontoutline=1, legend_fontweight='semibold',
           palette=list(sn.color_palette("pastel6").as_hex())+list(sn.color_palette("pastel").as_hex()),
          title='P22 Control:  4332 GABAergic Neurons', frameon=True,
          save='P22sham_gaba_FigS3.pdf')

## Markers for each type

In [ ]:
P22_glut = sc.read_h5ad('h5ads_cmprsd/P22Sham_glut_v3.h5ad')
P22_gaba = sc.read_h5ad('h5ads_cmprsd/P22Sham_gaba_v3.h5ad')

In [ ]:
def sub_marks(subclass):
    subclass_marks = []
    subclass = P22_glut[P22_glut.obs['Type'].str.startswith(subclass),:].copy()
    subclass = subclass[subclass.obs['Sample']!='S3',:]
    for i in subclass.obs['Type'].values.categories:
        subclass_marks.append(DE(subclass,obs_id='Type', obs_id_test=i, ref='rest', pts_thresh=0.25, lf_thresh=1))
    return subclass_marks, subclass

In [ ]:
def sub_marks_gab(subclass):
    subclass_marks = []
    subclass = P22_gaba[P22_gaba.obs['Type'].str.startswith(subclass),:].copy()
    subclass = subclass[subclass.obs['Sample']!='S3',:]
    for i in subclass.obs['Type'].values.categories:
        subclass_marks.append(DE(subclass,obs_id='Type', obs_id_test=i, ref='rest', pts_thresh=0.25, lf_thresh=1))
    return subclass_marks, subclass

In [ ]:
L4_marks, subclass = sub_marks('L4')

marks_plot = []
for i in L4_marks:
    marks_plot.append(i.index[0])
marks_plot.remove('Xkr6')
sc.pl.dotplot(subclass, var_names=['Nell1','Etv6', 'Gabrg3'], groupby='Type',
             save='_L4_marker.pdf')

In [ ]:
L23_marks, L23_sub = sub_marks('L2/3')

In [ ]:
marks_plot = []
for i in L23_marks:
    marks_plot.append(i.index[0])
    print(i.index[0:10])
sc.pl.dotplot(L23_sub, var_names=['Cdh13', 'Trpc6', 'Kcnh5'], groupby='Type',
             save='_L23_marker.pdf')

In [ ]:
L5_marks, L5_sub = sub_marks('L5')

In [ ]:
marks_plot = []
for i in L5_marks:
    marks_plot = marks_plot + list(i.index[0:3])
    print(i.index[0:10])

In [ ]:
print(marks_plot)

In [ ]:
sc.pl.dotplot(L5_sub, var_names=['Deptor', 'Dkk2', 
                                 'Svil', 'Pde7b', 'Cntnap5c', 
                                  'Erg', 'Gm44593', 
                                 'Col23a1', 'Egflam',], groupby='Type',
             save='_L5_marker.pdf')

In [ ]:
L6_marks, L6_sub = sub_marks('L6')

In [ ]:
marks_plot = []
for i in L6_marks:
    marks_plot = marks_plot + list(i.index[0:3])
    print(i.index[0:10])

In [ ]:
print(marks_plot)

In [ ]:
L6_marks[1].index

In [ ]:
sc.pl.dotplot(L6_sub, var_names=['Sema5a',  'Brinp3',
                                 
                                 
                                 'Trabd2b',
                                  'Bmpr1b', 'Epha6', 
                                 'Tafa1', 'Synpr', 'Inpp4b'], groupby='Type',
             save='_L6_marker.pdf')

In [ ]:
pv_marks, pv_sub = sub_marks('Pvalb')

In [ ]:
marks_plot = []
for i in pv_marks:
    marks_plot = marks_plot + list(i.index[0:4])
    print(i.index[0:10])

In [ ]:
print(marks_plot)

In [ ]:
'Col16a1', 'Il1rapl2', 'Zfpm2', 'Ralyl', 'Vipr2'

In [ ]:
sc.pl.dotplot(pv_sub, var_names=['Col16a1', 'Il1rapl2', 'Zfpm2', 'Ralyl', 'Vipr2']
              , groupby='Type', save='_Pvalb_marker.pdf')

In [ ]:
sst_marks, sst_sub = sub_marks('Sst')

In [ ]:
marks_plot = []
for i in sst_marks:
    marks_plot = marks_plot + list(i.index[0:4])
    print(i.index[0:10])

In [ ]:
print(marks_plot)

In [ ]:
'Hpse','Pcsk5','Plpp4','Prkg2','Grm8', 'Pard3b', 'Pdyn','Fbn2','Chodl'

In [ ]:
sst_sub.obs['Type'].values.categories

In [ ]:
sc.pl.dotplot(sst_sub, var_names=['Hpse','Pcsk5','Plpp4','Prkg2','Grm8', 
                                  'Pard3b', 'Pdyn','Fbn2','Chodl'], groupby='Type',
             save='_Sst_marker.pdf')

In [ ]:
Lamp5_marks, Lamp5_sub = sub_marks('Lamp5')

In [ ]:
marks_plot = []
for i in Lamp5_marks:
    marks_plot = marks_plot + list(i.index[0:4])
    print(i.index[0:10])

In [ ]:
print(marks_plot)

In [ ]:
sc.pl.dotplot(Lamp5_sub, var_names=['Htr1f', 
                                    'Ndnf', 
                                    'Cdh18',], 
              groupby='Type', save='_Lamp5_marker.pdf')

In [ ]:
Vip_marks, Vip_sub = sub_marks('Vip')

In [ ]:
marks_plot = []
for i in Vip_marks:
    marks_plot = marks_plot + list(i.index[0:4])
    print(i.index[0:10])

In [ ]:
print(marks_plot)

In [ ]:
'Ryr3', 'Sox5', 'Frem1', 'Grin3a' , 'Fbxo2'

In [ ]:
sc.pl.dotplot(Vip_sub, var_names=['Ryr3', 'Sox5', 'Frem1', 'Grin3a' , 'Fbxo2'], groupby='Type',
             save='_Vip_marker.pdf')

In [ ]:
L4v_marks = []
L4v = P21_V1[P21_V1.obs['Type'].str.startswith('L4'),:].copy()
for i in L4v.obs['Type'].values.categories:
    L4v_marks.append(DE(L4v,obs_id='Type', obs_id_test=i, ref='rest', pts_thresh=0.4, lf_thresh=1))

for i in L4v_marks:
    sc.pl.dotplot(L4v, var_names=i.index[0:20], groupby='Type')

In [ ]:
a = list(L4_marks[0].index[0:20])
a.remove('4933424G05Rik')
sc.pl.dotplot(L4, var_names=a, groupby='Type')
sc.pl.dotplot(L4v, var_names=a, groupby='Type')

In [ ]:
a = list(L4_marks[0].index[0:40])
a.remove('4933424G05Rik')
sc.pl.dotplot(L4, var_names=a, groupby='Type')
sc.pl.dotplot(L4v, var_names=a, groupby='Type')

In [ ]:
sc.pl.dotplot(L4, var_names=L4v_marks[0].index[0:20], groupby='Type')
sc.pl.dotplot(L4v, var_names=L4v_marks[0].index[0:20], groupby='Type')

sc.pl.dotplot(L4, var_names=L4v_marks[1].index[0:20], groupby='Type')
sc.pl.dotplot(L4v, var_names=L4v_marks[1].index[0:20], groupby='Type')

sc.pl.dotplot(L4, var_names=L4v_marks[2].index[0:20], groupby='Type')
sc.pl.dotplot(L4v, var_names=L4v_marks[2].index[0:20], groupby='Type')

### Save markers table

In [ ]:
L4_marks, L4_sub = sub_marks('L4')
L23_marks, L23_sub = sub_marks('L2/3')
L5_marks, L5_sub = sub_marks('L5')
L6_marks, L6_sub = sub_marks('L6')
pv_marks, pv_sub = sub_marks_gab('Pvalb')
sst_marks, sst_sub = sub_marks_gab('Sst')
Vip_marks, Vip_sub = sub_marks_gab('Vip')
Lamp5_marks, Lamp5_sub = sub_marks_gab('Lamp5')

In [ ]:
import pandas as pd

# Example dictionary where each item is a list of DataFrames
data_dict = {
    'L23 Type': L23_marks,
    'L4 Type': L4_marks,
    'L5 Types': L5_marks,
    'L6 Types': L6_marks,
    'Pvalb Types': pv_marks,
    'Sst Types': sst_marks,
    'Vip Types': Vip_marks,
    'Lamp5 Types': Lamp5_marks
    
}

# Create an Excel writer object
with pd.ExcelWriter('Tables/TableS3.xlsx') as writer:
    for sheet_name, df_list in data_dict.items():
        # Initialize row number for the start of each DataFrame
        startrow = 0
        
        # Iterate over the list of DataFrames
        for df in df_list:
            # Write the DataFrame to the Excel sheet
            df.columns =  [df.columns.tolist()[0]+' % Exp'] + df.columns.tolist()[1:]

            df.to_excel(writer, sheet_name=sheet_name, startrow=startrow, index=True)
            
            # Update the starting row for the next DataFrame
            startrow += len(df) + 2  # Adding 2 for some space between DataFrames

# The Excel file is saved as 'output.xlsx'
